### Preliminary setup

In [625]:
#Import gensim library
import gensim
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import PlaintextCorpusReader
import nltk
import pandas as pd
import numpy as np
import time
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter


### Load Dataset and Train Word2Vec model

In [626]:
start=time.time()
df = pd.read_csv("./amazon_food_review/Reviews.csv")
df.columns
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

Time taken 0.100616 min


### Extract Top Products with the most Reviews

In [627]:
 
top_product = Counter(df.ProductId).most_common()[0:1]

for i in range(len(top_product)):
    if i ==0:
        df2= df[df.ProductId == top_product[i][0]]
    else:
        df2 = pd.concat((df2,df[df.ProductId == top_product[i][0]]))
        
df2.shape

(913, 10)

### Check the count of Reviews in the dataset

In [628]:
from collections import Counter
print(Counter(df2.ProductId).most_common())
print(Counter(df2.Score))

[('B007JFMH8M', 913)]
Counter({5: 624, 4: 217, 3: 56, 2: 12, 1: 4})


### Tokenize the Reviews into Sentences - sent_tokenize

In [629]:
start=time.time()
#sents=list(df2.Text)
sents = df2.Text
sent_lst = []
for line in sents:
    #print(line)
    for token_line in sent_tokenize(line):
        #print(token_line)
        sent_lst.append([token_line])
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

len(sent_lst)

Time taken 0.003588 min


3667

### Sentences - Word_tokenize

In [630]:
from random import shuffle

start=time.time()
sent_lst2 = []
#unigram_lst=[]
for line in sent_lst:
    #print(word_tokenize(line[0]))
    sent_lst2.append(word_tokenize(line[0]))
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

shuffle(sent_lst2)
sent_train = sent_lst2[0: round(0.64*(len(sent_lst2)))] #SET 64% OF TRAINING DATA
len(sent_train)

Time taken 0.011879 min


2347

### Word2Vec Model

In [631]:
#Train the Word2Vec Model with 100 features
model = Word2Vec(sent_train)  

#Initialize the list of feature
feature_word_lst= []
for word in ["cookies","Cookies", "cookie", "Cookie"]: #Review Topics
    word_lst = list(model.most_similar(word, topn=100)) #Find top 100 most similar words
    for i in range(len(word_lst)):
        #print(word_lst[i][0])
        feature_word_lst.append(word_lst[i][0]) #Save similar to a list: feature_lst
    
print(set(feature_word_lst))            
feature_word_lst = list(set(feature_word_lst)) #Get the Union of all the feature words            

{'definitely', 'Cookies', 'their', 'does', 'right', 'sample', 'you', 'perfect', 'making', 'ever', 'buy', 'will', 'be', 'taste', 'been', 'not', "'m", 'Voxbox', ',', 'always', 'great', 'some', 'through', 'just', 'Raisin', 'being', 'had', 'must', 'daughter', 'has', 'fresh', 'could', 'used', 'because', 'better', 'cookie', 'more', 'can', 'delicious', 'the', 'try', 'VoxBox', 'which', 'package', 'Raisins', 'really', 'much', 'see', 'for', 'when', 'big', 'tasted', 'half', 'they', 'in', 'getting', 'after', 'she', 'am', 'into', 'Baked', 'bag', 'our', 'go', 'chance', 'raisins', 'snack', 'Oatmeal', 'husband', 'before', 'grain', 'moist', 'family', 'favorite', 'store', 'then', 'tastes', 'given', 'Very', 'than', 'yummy', 'also', 'super', 'got', 'never', 'part', 'were', 'full', 'son', 'I', 'Cookie', 'how', 'them', 'ones', 'most', 'we', 'flavor', 'products', '!', 'make', 'makes', 'recently', 'oatmeal', 'was', 'quick', 'so', 'that', 'even', 'healthy', 'this', 'bought', 'as', 'it', 'out', 'my', 'Quaker', 

/Users/KevQuant/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


### Step-4: hand-write some patterns

In [632]:
#regex grammar
grammar = r"""
JN:{<RB.*|JJ.*>+<.>*}

NP:{<EX|DT|PRP\$>?<RB>*<JN|VB.*>+<NN.*>+<.>*}

cla1a:{<DT|PRP>?<NN.*>+<VB.*>+<JN>+<JJ.*>*<.>*}
        
cla1b:{<cla1a>+<,|CC>+<JN|NN.*>+<JN|NN.*>*<.>*}
        {<cla1a>+<,|CC>+<cla1a>+<.>*}

cla2a:{<DT>?<NN.*|PRP.*>+<IN|VB.*>+<DT>*<NP|NN.*>+<NP|NN.*|JN>*<.>*}

cla2b:{<DT>?<cla2a>+<,|CC>+<DT>*<JN|NN.*|NP>+<.>*}
        {<DT>?<cla2a>+<,|CC>+<cla2a>+<.>*}
                
cla3: {<DT>?<NN.*>+<NP>+<JN>*<.>*<IN>*<NP>*}

cla4: {<cla1a>+<VB.*>+<JN>*<.>*}

cla4b: {<cla4>+<PRP.*>+<NN.*>+<TO>*<VB.*>*<.>*}

cla5: {<NN.*|PRP.*><NP>+<IN>+<NP>+<.>*}
            {<NN.*|PRP.*><NP>+<IN>+<NP>*<.>*}

cla6: {<PRP>*<MD>*<JN>*<VB.*>+<DT>+<TO>+<NP>+<.>*}

cla7: {<WP>+<PRP>*<MD>+<JN>*<VB.*>+<PRP>+<JN>+<.>*}

cla8: {<PRP>?<MD>*<JN>*<VB.*>+<DT>+<TO>+<PRP\$>+<NN.*>+<.>*}

cla9: {<DT>*<NN>+<MD>*<VB.*>+<DT>+<NN>+<JN>+<IN>+<CD>*<NN.*>*<.>*}

cla10: {<PRP>+<VB.*>+<JN>+<NP>+<IN>+<NP+>*<.>*}

cla11: {<CC>+<DT>*<NN.*>?<JN>+<VB.*>+<IN>+<PRP.*>*<NN>*<.>*}

cla12: {<CC>?<VB.*>+<IN>+<PRP.*>+<.>*}

cla13: {<CC>?<PRP.*>+<VB.*>+<JN>+<IN>+<DT|VB.*>*<JJ.*|JN|NN.*>*<.>*}

cla14:{<CC>?<PRP.*>*<MD>*<JN>*<VB.*><TO|IN>+<VB.*|JN>+<.>*}

cla15: {<CC>?<NN.*|PRP>+<VB.*>+<VB.*|JN>+<PRP.*|DT|WDT|NN>*<VB.*>*<.>*}

cla15b: {<cla15>+<CC>+<JN|JJ>+<.>*}

cla15c: {<cla15>+<TO>+<VB.*>+}

cla16:{<CC>?<DT>+<VB.*>*<JN|JJ.*|NP>+<JN|VB.*>*<.>*}

cla16b:{<cla16><,|CC>+<NN.*|NP>*<.>*}

cla16c:{<cla16b>+<PRP.*>+<JN>*<VB.*>+<.>*}

cla17: {<NP>+<WDT|WP.|WRB>*<PRP.*>*<VB.*>+<NP|JN>*<.>*}

cla17b: {<cla17>+<TO|VBG>+<VB.*>*<JN>*<.>*}

cla17c:{<cla17>+<PRP.*>+<JN | JJ>*}

cla18:{<DT>?<CD>+<IN>*<JN|NN.*>*}

cla18a:{<cla18>+<,|CC>+<cla18>+<.>*}

cla19: {<PRP.*>*<JN>*<WP>?<MD>*<VB.*>+<DT|PRP.*>*<NP|NN.*|PRP.*>+<.>*}

cla19b: {<cla18>+<cla19>+<.>*}

cla20: {<cla15>+<cla12>+<.>*}

cla21: {<PRP>*<MD>*<JN>+<TO>*<VB>+<DT>*<cla16|cla18>*}

cla22: {<VB.*>+<cla2a>+<WRB|WP|WDT>*<PRP>*<VB.*>*<.>*}

cla23:{<cla2a>+<WDT|WP>+<cla19>+}

cla24: {<NNP>+<JN>+<IN>+<DT>+<PRP>*<:|.>*}

cla24b:{<cla24>+<cla17>+}

cla25: {<NN.*>+<TO|PRP.*>*<NN.*>+<,|IN|CC>*<NN.*>*<IN|TO>*<VB.*>+<PRP.*|DT>*<:|.>*}

cla25b:{<cla25>+<,|CC>*<JN|NP>+<.|:>*}

cla26: {<PRP.*>+<NN.*>+<,|IN|CC>*<NN.*>*<VB.*>+<DT|PRP.*>*<JN>*<:|.>*}

cla27: {<cla15>+<IN>*<WRB|WDT|WP.*>*<JN>+<DT|NN.*>*<VB.*>+<:|.>*}

cla28:{<NP|NNP>+<,|CC>*<NP|JN>+}

cla29: {<WP>+<JN>*<PRP.*>+<MD>+<VB.*>*<:|.>*}
    
cla30: {<WP>+<JN>*<VB.*>+<cla21>+<:|.>*}

cla31: {<NN.*>+<,|CC|IN>*<JN|JJ|NN.*>*<,|CC|IN>*<NN.*>*<:|.>*}

cla32: {<NN.*|PRP.*>*<MD>+<VB.*>*<PRP.*|DT>+<JN>*<.|:>*}

cla33:{<cla19>+<DT|JN | RP>*<.|:>*}

cla34:{<PRP.*>+<cla18>+<cla3>*<cla19>+<JN>*<.|:>*}

cla35:{<DT>*<JJS>+<,|CC>*<JJS>*<NN.*><PRP.*|DT>+<JN>*<.|:>*}

cla36:{<cla2a>+<IN>*<DT>*<NN.*|NP|cla31>+<.|:>*}

cla37:{<DT|PRP.*>*<cla15>+<CC|IN|,>*<cla15>+<.|:>*}

cla38:{<DT>+<MD>+<VB.*>+<IN|DT>*<cla31>+<.|:>*}

cla39:{<NN.*|PRP.*>+<VB.*>*<PRP.*>+<JN>*<.|:>*}

cla40:{<NN.*|PRP.*>*<MD>*<VB.*>+<JN|cla16>+<.|:>*}

cla41:{<cla33>+<TO|IN>+<cla2a>+<.|:>*}

cla42:{<cla31>+<TO|IN>+<VB.*>+<TO|IN>+<NP>+<.|:>*}

cla43:{<DT>*<cla31>+<cla40>+<TO|IN>+<DT|PRP.*>*<cla31>*<.|:>*}

cla44:{<WP>+<cla40>+<IN>*<DT|PRP.*>+<NP|NN.*>+<.|:>*}
    
cla44b:{<WP>+<cla40>+<IN>+<cla18|DT|NN.*|cla31>+<.|:>*}

cla45:{<cla2b>+<cla40>+<.|:>*}

cla46:{<cla13>+<NP>+<.|:>*}

cla47:{<WRB>+<JN|VB.*>+<CC|IN>*<JN|VB.*>*<DT|PRP.*>*<VB.*>+<.|:>*}

cla48:{<cla17>+<IN>+<cla2a>+<.|:>*}

cla49:{<cla19>+<IN>+<PRP.*>*<cla18|cla31>+<.|:>*}

cla49b: {<cla49>+<cla3>+<.|:>*}

cla50: {<PRP.*>?<cla12>+<cla31>+<.|:>*}

cla51:{<CC>*<DT|PRP.*>*<cla31>+<cla19+>+<JN>*<.|:>*}

cla52:{<cla2a>+<MD>*<VB.*>+<WRB>*<JN>+<DT>*<PRP.*|cla31>*<VB.*>*<.|:>*}

cla53:{<cla31>+<cla39|VG.*>*<DT|cla31>*<cla33>+<.|:>*}

"""

In [633]:
############# Grammar: NP
#sent = "Nicely smell cookiess"
#sent = "tasty cookies"


############ cla1a
#sent = "The cookiess seemed pretty tasty"
#sent = "the cookiess were excellent"
#sent = "the cookies is very nice"
#sent = "the bottles are not too large"
#sent = "the bottles are too large"
#sent ="the cookies are not good"
#sent ="the cookies are not very good enough"
#sent ="the cookies are not very good"
#sent = "this cookie was surprisingly yummy."

############ cla1b
#sent = "cookies was tasty and soft"
#sent = " the cookies is pretty beautiful and the cookies is pretty soft"
#sent = " the cookies is big and the rasins is very soft"

############ cla2a
#sent="I love the food so much"
#sent = "I participated in a product review"


############ cla2b
#sent = "She likes this big cookie and soft cookie"
#sent = "She likes this cookies and the smell"


############cla3
#sent = "the cookies are way too small"
#sent = "The cookies are soft, chewy, yummy,"
#sent = "These cookies have perfect flavor, excellent moist/dry combination"

############cla4
#sent="the cookie did not smell good"


############ cla4b
#sent="please don't waste your money "

############cla5
#sent = "I love cookies with many rasins"


########### cla7
#sent= "who would buy it again??"


############ cla13
#sent = "It goes down hill from there."
#sent = "They're are so soft, perfect for anytime"


############ cla14
#sent = "I can't wait to try "
#sent="can't wait to buy more"

############ cla15
#sent="It was great!!!"
#sent = "i love eating it"
#sent = "They're are so soft and perfect"

############ 15b
#sent="They are so soft and tasty!"

############ 15c
#sent = "it is great to buy it"

########## cla16
#sent = "This is my great best experience!"
#sent = "This is the high point."
#sent = "This was the most delightful cookie i ever tasted."

########## cla16b
#sent = "the best and freshest cookie"

########## cla16c
#sent = "the best and freshest cookie I ever had"

######### cla#17
#sent = "this is bad food which hurt your healthy body"
#sent = "The high quality ingredients are a crowd pleaser."

##########17c
#sent = "Oatmeal cookies are my favorite"

########## cla18
#sent="6g of fat , 27g total carbs"

########## cla19
#sent="what's the point?"
#sent="you'll love this snack ....."
#sent = "I highly recommend this cookie"
#sent = "loved the food"
#sent = "loved them"
#sent ="I love my cookies,"

######### cla21
#sent = "too old to eat"
#sent="I would never try one again."
#sent="I would never try this one again."

######## cla22
#sent="throw them in the garbage which they belong"
#sent="throw them in the garbage where they belong"

######## cla23
#sent="I participated in a product review that included a sample"

######## cla26
#sent="My kids and i  loved these."


######## cla27
#sent="I was surprised how soft the cookie was."
#sent="I was immediately surprised by how soft the cookie was "

####### cla#28
#sent = "Delicious and nutritious ....."
#sent = "good price and delicious"

######## cla29
#sent="what else you can say!!!"

######## cla30
#sent="what else is there to say...."

####### cla31
#sent = "Soft, chewy and tasty."
#sent = "Highly recommend!"
#sent = "yummy and delicious."

####### cla32
#sent="I would recommend these"

####### cla33
#sent = "we ate them all...!!!"
#sent = "we ate them up...!!!"
#sent= "should i buy it again??"

###### cla34
#sent="My three year old kid loved it so much !!!"


###### cla36
#sent = "I am a huge fan of the Quaker Chewy cookies!"
#sent = "I am a huge fan of the cookies!"
#sent = "I am a huge fan of the Chewy cookies!"
#sent = "I am a huge fan of Chewy cookies!"
#sent = "I am a huge fan of cookies!"

###### cla37
#sent="The moist cookies are satisfying and they are not too sweet."

###### cla40
#sent ="I will buy some more"

###### cla41
#sent = "I recommend these to anyone with a sweet tooth"


###### cla42
#sent = "Great to take as a quick breakfast "

###### cla43
#sent = "This cookie would be perfect for breakfast"
#sent = "This cookie would be perfect for my family..."
#sent = "This cookie would be perfect for me"

###### cla44
#sent = "What could be better than that?!"

###### cla44b
#sent = "What could be better than this one ?!"
#sent = "What could be better than this product ?!"
#sent = "What could be better than these products ?!"
#sent = "What could be better than that?!"

###### cla46
#sent = "they are more healthy than most other products."
#sent="they are more healther than other food"

###### cla47
#sent = "how soft and fresh it tasted"
#sent="How good it tasted?"

###### cla49
#sent="I even shared it with my brother"
#sent="I shared it with my child"

####### cla50
#sent = "They taste like my grandmas homeade cookies.."

######## cla51
#sent = "the kids love them!"
#sent="My  kid loved it so much "

######## cla52
#sent = "I guess the younger folks will like how sweet the cookie is."

######## cla53
#sent="Thank you my mom buying me this!!!"
#sent="thanks for buying this"
#sent="Thanks for buying me this"


#print(sent)
#print()
#print(nltk.pos_tag(nltk.word_tokenize(sent)))
#print()
#sent =  nltk.word_tokenize(sent)
#print(sent)
review_lst=[]


for sent in sent_train:
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":        print("NP", subtree.leaves()),review_lst.append(subtree.leaves())
        #if subtree.label() =="JN":         print("JN", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="Quotation":   print("Quotation", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1a":         print("cla1a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1b":         print("cla1b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla2a":         print("cla2a", subtree.leaves()), review_lst.append(subtree.leaves())
        if subtree.label() =="cla2b":         print("cla2b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla3":         print("cla3", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4":         print("cla4", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4b":         print("cla4b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla5":         print("cla5", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla6":         print("cla6", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla7":         print("cla7", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla8":         print("cla8", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla9":         print("cla9", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla10":         print("cla10", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla11":         print("cla11", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla12":         print("cla12", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla13":         print("cla13", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla14":        print("cla14", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15":         print("cla15", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15b":         print("cla15b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15c":         print("cla15c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16":         print("cla16", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16b":         print("cla16b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16c":         print("cla16c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17":         print("cla17", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17b":         print("cla17b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17c":         print("cla17c", subtree.leaves()),review_lst.append(subtree.leaves())        
        if subtree.label() =="cla18":         print("cla18", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla18a":         print("cla18a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19":         print("cla19", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19b":         print("cla19b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla20":         print("cla20", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla21":         print("cla21", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla22":         print("cla22", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla23":         print("cla23", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24":         print("cla24", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24b":         print("cla24b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25":         print("cla25", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25b":         print("cla25b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla26":         print("cla26", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla27":         print("cla27", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla28":         print("cla28", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla29":         print("cla29", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla30":         print("cla30", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla31":         print("cla31", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla32":         print("cla32", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla33":         print("cla33", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla34":         print("cla34", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla35":         print("cla35", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla36":         print("cla36", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla37":         print("cla37", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla38":         print("cla38", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla39":         print("cla39", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla40":       print("cla40",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla41":       print("cla41",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla42":       print("cla42",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla43":       print("cla43",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44":       print("cla44",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44b":       print("cla44b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla45":       print("cla45",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla46":       print("cla46",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla47":       print("cla47",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla48":       print("cla48",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49":       print("cla49",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49b":       print("cla49b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla50":       print("cla50",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla51":       print("cla51",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla52":       print("cla52",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla53":       print("cla53",subtree.leaves()),review_lst.append(subtree.leaves())
print("Number of Extracted Segments",len(review_lst))

cla2b [('I', 'PRP'), ('found', 'VBD'), ('these', 'DT'), ('cookies', 'NNS'), ('really', 'RB'), ('moist', 'JJ'), ('and', 'CC'), ('yummy', 'NN'), ('.', '.')]
cla2a [('I', 'PRP'), ('found', 'VBD'), ('these', 'DT'), ('cookies', 'NNS'), ('really', 'RB'), ('moist', 'JJ')]
cla53 [('kids', 'NNS'), ('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
cla31 [('kids', 'NNS')]
cla33 [('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
cla19 [('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
cla1a [('These', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('absolutely', 'RB'), ('delicious', 'JJ'), (',', ',')]
cla18 [('one', 'CD'), ('for', 'IN')]
cla2a [('review', 'NN'), ('purposes', 'NNS'), ('from', 'IN'), ('Influenster.com', 'NNP'), ('.', '.')]
cla2b [('I', 'PRP'), ('RECEIVED', 'VBP'), ('A', 'DT'), ('FREE', 'NNP'), ('COOKIE', 'NNP'), ('FROM', 'NNP'), ('INFLUENSTER', 'NNP'), ('AND', 'CC'), ('HE', 'NNP'), ('TOOK', 'NNP'), ('IT', 'NNP'), (',', ',')]
cla2a

cla15 [('I', 'PRP'), ("'m", 'VBP'), ('more', 'RBR'), ('used', 'JJ')]
cla13 [('them', 'PRP'), ('being', 'VBG'), ('hard', 'JJ'), ('so', 'IN')]
cla33 [('I', 'PRP'), ('really', 'RB'), ('liked', 'VBD'), ('the', 'DT'), ('soft', 'JJ'), ('chewiness', 'NN')]
cla19 [('I', 'PRP'), ('really', 'RB'), ('liked', 'VBD'), ('the', 'DT'), ('soft', 'JJ'), ('chewiness', 'NN')]
NP [('the', 'DT'), ('soft', 'JJ'), ('chewiness', 'NN')]
cla31 [('cookie', 'NN'), ('.', '.')]
cla32 [('I', 'PRP'), ('will', 'MD'), ('be', 'VB'), ('buying', 'VBG'), ('some', 'DT')]
NP [('Very', 'RB'), ('great', 'JJ'), ('product', 'NN')]
cla31 [('money', 'NN'), ('!', '.')]
cla3 [('Great', 'NNP'), ('cinnamon', 'JJ'), ('flavor', 'NN'), ('.', '.')]
NP [('cinnamon', 'JJ'), ('flavor', 'NN'), ('.', '.')]
cla15 [('They', 'PRP'), ('are', 'VBP'), ('very', 'RB'), ('soft', 'JJ')]
NP [('actually', 'RB'), ('taste', 'NN')]
cla31 [('homemade', 'NN'), ('cookies', 'NNS')]
cla31 [('grandma', 'NNS')]
cla15 [('It', 'PRP'), ('was', 'VBD'), ('mouth', 'RB'), 

cla2a [('I', 'PRP'), ('love', 'VBP'), ('these', 'DT'), ('cookies', 'NNS')]
cla15b [('they', 'PRP'), ('are', 'VBP'), ('soft', 'JJ'), ('and', 'CC'), ('good', 'JJ'), ('!', '.')]
cla15 [('they', 'PRP'), ('are', 'VBP'), ('soft', 'JJ')]
cla1a [('This', 'DT'), ('snack', 'NN'), ('is', 'VBZ'), ('great', 'JJ')]
NP [('the', 'DT'), ('entire', 'JJ'), ('family', 'NN'), (',', ',')]
cla22 [('knowing', 'VBG'), ('we', 'PRP'), ("'re", 'VBP'), ('making', 'VBG'), ('a', 'DT'), ('healthier', 'JJR'), ('choice', 'NN')]
cla2a [('we', 'PRP'), ("'re", 'VBP'), ('making', 'VBG'), ('a', 'DT'), ('healthier', 'JJR'), ('choice', 'NN')]
NP [('a', 'DT'), ('healthier', 'JJR'), ('choice', 'NN')]
cla33 [('is', 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
cla19 [('is', 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
NP [('a', 'DT'), ('good', 'JJ'), ('feeling', 'NN'), ('.', '.')]
cla15b [('they', 'PRP'), ("'re", 'VBP'), ('really', 'RB'), ('healthy', 'JJ'), ('and', 'CC'), ('good', 'JJ

cla47 [('how', 'WRB'), ('satisfied', 'JJ'), ('I', 'PRP'), ('was', 'VBD'), ('.', '.')]
cla15 [('but', 'CC'), ('I', 'PRP'), ('did', 'VBD'), ('not', 'RB'), ('notice', 'VB')]
NP [('any', 'DT'), ('funny', 'JJ'), ('taste', 'NN')]
cla31 [('smell', 'NN')]
cla31 [('cookies', 'NNS'), ('.', '.')]
cla16 [('These', 'DT'), ('are', 'VBP'), ('delicious', 'JJ'), (',', ',')]
cla31 [('question', 'NN'), ('.', '.')]
NP [('a', 'DT'), ('very', 'RB'), ('hint', 'NN')]
cla31 [('molasses', 'NNS')]
cla21 [('too', 'RB'), ('old', 'JJ'), ('to', 'TO'), ('use', 'VB')]
NP [('were', 'VBD'), ('expecting', 'VBG'), ('something', 'NN')]
cla16 [('a', 'DT'), ('little', 'RB'), ('more', 'RBR'), ('.', '.')]
cla2a [('I', 'PRP'), ('was', 'VBD'), ('bugged', 'VBN'), ('for', 'IN'), ('days', 'NNS')]
cla2a [('her', 'PRP$'), ('for', 'IN'), ('another', 'DT'), ('cookie', 'NN')]
cla33 [('she', 'PRP'), ('must', 'MD'), ('have', 'VB'), ('loved', 'VBN'), ('it', 'PRP')]
cla19 [('she', 'PRP'), ('must', 'MD'), ('have', 'VB'), ('loved', 'VBN'), ('

cla53 [('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('Cookies', 'NNPS'), ('may', 'MD'), ('be', 'VB'), ('the', 'DT'), ('best', 'JJS'), ('oatmeal', 'JJ'), ('cookies', 'NNS'), ('you', 'PRP')]
cla31 [('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('Cookies', 'NNPS')]
cla33 [('may', 'MD'), ('be', 'VB'), ('the', 'DT'), ('best', 'JJS'), ('oatmeal', 'JJ'), ('cookies', 'NNS'), ('you', 'PRP')]
cla19 [('may', 'MD'), ('be', 'VB'), ('the', 'DT'), ('best', 'JJS'), ('oatmeal', 'JJ'), ('cookies', 'NNS'), ('you', 'PRP')]
NP [('the', 'DT'), ('best', 'JJS'), ('oatmeal', 'JJ'), ('cookies', 'NNS')]
cla40 [('can', 'MD'), ('eat', 'VB'), ('that', 'DT'), ('are', 'VBP'), ('not', 'RB'), ('homemade', 'VBN'), ('.', '.')]
cla16 [('that', 'DT'), ('are', 'VBP'), ('not', 'RB'), ('homemade', 'VBN'), ('.', '.')]
cla31 [('oatmeal', 'NN'), ('cookie', 'NN'), (',', ',')]
cla15 [('they', 'PRP'), ('are', 'VBP'), ("n't", 'RB'), ('that', 'RB'), ('good', 'JJ'), ('.', '.')]
NP [('My', 'PRP$'), ('only', 'JJ'), ('

...

cla21 [('I', 'PRP'), ('will', 'MD'), ('definitely', 'RB'), ('purchase', 'VB')]
cla16b [('these', 'DT'), ('again', 'RB'), ('and', 'CC')]
cla16 [('these', 'DT'), ('again', 'RB')]
cla33 [('try', 'VB'), ('the', 'DT'), ('other', 'JJ'), ('flavors', 'NNS'), ('.', '.')]
cla19 [('try', 'VB'), ('the', 'DT'), ('other', 'JJ'), ('flavors', 'NNS'), ('.', '.')]
NP [('the', 'DT'), ('other', 'JJ'), ('flavors', 'NNS'), ('.', '.')]
cla15 [('I', 'PRP'), ('was', 'VBD'), ('so', 'RB'), ('disappointed', 'JJ')]
cla12 [('understand', 'VBP'), ('from', 'IN'), ('him', 'PRP'), ('it', 'PRP')]
cla18 [('one', 'CD'), ('of', 'IN')]
NP [('the', 'DT'), ('best', 'JJS'), ('cookies', 'NNS')]
cla15 [('he', 'PRP'), ('has', 'VBZ'), ('ever', 'RB'), ('had', 'VBN'), (',', ',')]
cla13 [('and', 'CC'), ('it', 'PRP'), ('tastes', 'VBZ'), ('better', 'RBR'), ('than', 'IN'), ('the', 'DT'), ('cookies', 'NNS')]
cla22 [('bake', 'VBP'), ('him', 'PRP'), ('at', 'IN'), ('home', 'NN'), ('.', '.')]
cla2a [('him', 'PRP'), ('at', 'IN'), ('home', 'NN

cla31 [('cookies', 'NNS')]
cla50 [('be', 'VB'), ('found', 'VBN'), ('in', 'IN'), ('my', 'PRP$'), ('house', 'NN'), ('from', 'IN'), ('now', 'RB'), ('on', 'IN'), ('.', '.')]
cla12 [('be', 'VB'), ('found', 'VBN'), ('in', 'IN'), ('my', 'PRP$')]
cla31 [('house', 'NN'), ('from', 'IN'), ('now', 'RB'), ('on', 'IN'), ('.', '.')]
cla33 [('getting', 'VBG'), ('this', 'DT'), ('cookie', 'NN'), ('I', 'PRP'), ('ate', 'VBP'), ('it', 'PRP')]
cla19 [('getting', 'VBG'), ('this', 'DT'), ('cookie', 'NN'), ('I', 'PRP')]
cla19 [('ate', 'VBP'), ('it', 'PRP')]
cla1a [('These', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('so', 'RB'), ('delicious..', 'JJ')]
cla16 [('These', 'DT'), ('are', 'VBP'), ('a', 'DT'), ('great', 'JJ'), ('product', 'NN')]
NP [('a', 'DT'), ('great', 'JJ'), ('product', 'NN')]
NP [('keep', 'VB'), ('handy', 'NN')]
NP [('snack', 'JJ'), ('time', 'NN'), ('!', '.')]
cla2a [('I', 'PRP'), ('shared', 'VBD'), ('the', 'DT'), ('cookie', 'NN')]
cla31 [('preschool', 'NN'), ('daughter', 'NN'), ('.', '.')]
cla

cla25 [('Great', 'NNP'), ('snack', 'NN'), ('to', 'TO'), ('have', 'VB')]
cla31 [('cupboard', 'NN')]
cla28 [('want', 'VBP'), ('something', 'NN'), ('quick', 'JJ')]
NP [('want', 'VBP'), ('something', 'NN')]
cla17 [('already', 'RB'), ('loved', 'VBD'), ('Quaker', 'NNP'), ('products', 'NNS'), (',', ','), ('I', 'PRP'), ('grew', 'VBD')]
NP [('already', 'RB'), ('loved', 'VBD'), ('Quaker', 'NNP'), ('products', 'NNS'), (',', ',')]
cla33 [('is', 'VBZ'), ('my', 'PRP$'), ('dad', 'NN')]
cla19 [('is', 'VBZ'), ('my', 'PRP$'), ('dad', 'NN')]
cla31 [('lunch', 'NN'), ('because', 'IN')]
cla28 [('always', 'RB'), ('wants', 'VBZ'), ('something', 'NN'), ('sweet', 'JJ')]
NP [('always', 'RB'), ('wants', 'VBZ'), ('something', 'NN')]
cla33 [('I', 'PRP'), ('want', 'VBP'), ('him', 'PRP')]
cla19 [('I', 'PRP'), ('want', 'VBP'), ('him', 'PRP')]
cla40 [('eat', 'VB'), ('healthy', 'JJ'), ('.', '.')]
cla33 [('She', 'PRP'), ('loved', 'VBD'), ('it', 'PRP'), ('.', '.')]
cla19 [('She', 'PRP'), ('loved', 'VBD'), ('it', 'PRP'), (

cla5 [('they', 'PRP'), ('taste', 'VBP'), ('too', 'RB'), ('bland', 'NN'), ('for', 'IN')]
NP [('taste', 'VBP'), ('too', 'RB'), ('bland', 'NN')]
NP [('are', 'VBP'), ('indeed', 'RB'), ('quite', 'RB'), ('plain', 'NN'), ('.', '.')]
cla31 [('o', 'NN'), (')', ')')]
cla31 [('Highly', 'NNP'), ('recommend', 'NN'), ('and', 'CC')]
cla21 [('I', 'PRP'), ('will', 'MD'), ('definitely', 'RB'), ('be', 'VB')]
cla40 [('buying', 'VBG'), ('this', 'DT'), ('again', 'RB'), ('!', '.')]
cla16 [('this', 'DT'), ('again', 'RB'), ('!', '.')]
cla2a [('my', 'PRP$'), ('Mom', 'NNP'), ('VoxBox', 'NNP'), ('from', 'IN'), ('Influenster', 'NNP'), ('.', '.')]
cla36 [('I', 'PRP'), ('had', 'VBD'), ('a', 'DT'), ('problem', 'NN'), ('with', 'IN'), ('the', 'DT'), ('crumbly', 'NN'), ('texture', 'NN'), (',', ','), ('because', 'IN')]
cla2a [('I', 'PRP'), ('had', 'VBD'), ('a', 'DT'), ('problem', 'NN')]
cla31 [('crumbly', 'NN'), ('texture', 'NN'), (',', ','), ('because', 'IN')]
cla14 [('I', 'PRP'), ('tried', 'VBD'), ('to', 'TO'), ('dip',

cla15 [('She', 'PRP'), ('was', 'VBD'), ('really', 'RB'), ('bummed', 'VBN'), ('.', '.')]
cla28 [('Quaker', 'NNP'), ('Breakfast', 'NNP'), ('Cookies', 'NNP'), ('Oatmeal', 'NNP'), ('Raisin', 'NNP'), (',', ','), ('6-Count', 'JJ'), ('Boxes,10.1-Ounces', 'NNS'), ('(', '(')]
NP [('6-Count', 'JJ'), ('Boxes,10.1-Ounces', 'NNS'), ('(', '(')]
cla31 [('Pack', 'NNP'), ('of', 'IN')]
cla18 [('6', 'CD')]
cla2a [(']', 'NN'), (']', 'NNP'), ('s.', 'NN'), ('They', 'PRP'), ('come', 'VBP'), ('in', 'IN'), ('individually', 'RB'), ('wrapped', 'VBN'), ('packages', 'NNS')]
NP [('individually', 'RB'), ('wrapped', 'VBN'), ('packages', 'NNS')]
cla22 [('use', 'VB'), ('them', 'PRP'), ('as', 'IN'), ('snacks', 'NNS')]
cla2a [('them', 'PRP'), ('as', 'IN'), ('snacks', 'NNS')]
cla31 [('school', 'NN'), (',', ','), ('if', 'IN')]
cla47 [('how', 'WRB'), ('soft', 'JJ'), ('and', 'CC'), ('chewy', 'VB')]
cla17 [('this', 'DT'), ('oatmeal', 'JJ'), ('cookie', 'NN'), ('was', 'VBD'), ('.', '.')]
NP [('this', 'DT'), ('oatmeal', 'JJ'), (

### To CSV

In [634]:
Ext_sen_df = pd.DataFrame(review_lst)
Ext_sen_df.to_csv("Ext_sen.csv", index=False, header=False)


### Filtering out all the segments without Adverb or Adjectives or Verbs

In [664]:
review_lst2=[]
for tups in review_lst:
#for tups in test_lst:
    #print(tups)
    filter_cnt=0
    for tup in tups:
        if tup[1] in ["JJ","JJR","JJS","RB","RBR","RBS","VB","VBD","VBG","VBN","VBP", "VBZ"]:
            #print(tup[1])
            filter_cnt+=1
    
    if filter_cnt>0:
        review_lst2.append(tups)
print("Number of Extracted Segments after filtering",len(review_lst2))

Number of Extracted Segments after filtering 7234


### Filtering out Segments not including Feature words

In [665]:
review_lst3=[]
for i in range(len(review_lst2)):
    for j in range(len(review_lst2[i])):
        if review_lst2[i][j][0] in feature_word_lst:
            review_lst3.append(review_lst2[i])
            #print()
            break
        
#print(len(Y_true)        )
#print(len(X_true)        )
print("Number of Extracted Segments after filtering",len(review_lst3))

Number of Extracted Segments after filtering 6866


### Functions to get the Wordnet Tag  and lemmatized words forms

In [666]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

def to_wntag(pos):
    """
    Given a Penn tag, returns the corresponding Wordnet pos tag.
    Note that WordNet only contain nouns/verbs/adjectives/adverbs.
    """
    ch = pos[0]
    if ch == 'N':
        return wn.NOUN #noun
    elif ch == 'V':
        return wn.VERB # verb
    elif ch == 'J':
        return wn.ADJ # adjective
    elif ch == 'R':
        return wn.ADV # adverb
    else:
        return None

def wn_lemmatize(token, wntag):
    """
    Applies WordNet morpher (which requires a pos) and 
    returns the morphological lemma form, or None.
    """
    return wn.morphy(token, wntag)

### Step-5: Parse segments to get the Synset[0] and Synset[ALL]

In [667]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
#df_lst = [0 for item in range(len(review_lst2))]
df_syn0_lst = []
df_syn_all_lst = []
df_tup_lst = []

cnt=0
#for tups in review_lst[0:10]:
for tups in review_lst3:
    if len(tups)<3:
        continue
    #print(tups)
    lst=[]
    tot_pscore_syn0=0
    tot_pscore_syn_all = 0
    word_cnt=0
    
    for i in range(len(tups)):
        #print("tups[i]",tups[i])
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        #print("w",w)
        #print("tag",tag)
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print("1")
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                #print("2")
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                #print("3")
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            #print(synsets)
            #print(synset)
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            #print(swn_synset)
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            #print(pscore)
            tot_pscore_syn0+=pscore
            
            tot_pscore_syn=0
            if len(synsets)>1:
                for synset in synsets:
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    #print(swn_synset)
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    #print("pscore",pscore)
                    tot_pscore_syn += pscore
                tot_pscore_syn_all += (tot_pscore_syn/len(synsets))
                #print("tot_pscore_syn_all",tot_pscore_syn_all)
            else: tot_pscore_syn_all += tot_pscore_syn0

            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
    
    if word_cnt == 0:
        df_syn0_lst.append(0)
        df_syn_all_lst.append(0)
        #print("Word count",word_cnt)
        df_tup_lst.append(tups)
        cnt+=1
    else:
        df_syn0_lst.append(tot_pscore_syn0/word_cnt)
        df_syn_all_lst.append(tot_pscore_syn_all/word_cnt)
        #print("Word count",word_cnt)
        df_tup_lst.append(tups)
        cnt+=1
        
print(len(df_syn0_lst), len(df_syn_all_lst), len(df_tup_lst))
    


6375 6375 6375


In [668]:
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
pd.DataFrame(list(zip(df_tup_lst,df_syn0_lst,df_syn_all_lst )), columns=["segments","syn0", "syn_all"])
#df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
#df2= pd.DataFrame(df_lst2)
#df = pd.concat((df1,df2), axis=1)
#df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 

#                   ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

,segments,syn0,syn_all
0,"[(i, PRP), (found, VBD), (these, DT), (cookies...",0.125000,0.201316
1,"[(i, PRP), (found, VBD), (these, DT), (cookies...",0.156250,0.251645
2,"[(kids, NNS), (also, RB), (loved, VBD), (the, ...",0.125000,0.154688
3,"[(also, RB), (loved, VBD), (the, DT), (cookies...",0.166667,0.197917
4,"[(also, RB), (loved, VBD), (the, DT), (cookies...",0.166667,0.197917
5,"[(these, DT), (cookies, NNS), (are, VBP), (abs...",0.416667,0.375000
6,"[(i, PRP), (received, VBP), (a, DT), (free, NN...",0.000000,0.017308
7,"[(i, PRP), (received, VBP), (a, DT), (free, NN...",0.000000,0.021635
8,"[(i, PRP), (told, VBD), (him, NNP), (he, NNP),...",0.041667,0.115972
9,"[(slow, JJ), (to, TO), (eat, VB)]",0.000000,-0.062500


In [259]:
Counter(df_tup_lst[0]).most_common()[0][]

1

### Featuring process from the Segments (e.g. Word structures, Part of Speech)

In [669]:
repeat_word_cnt_lst=[]#1
no_cnt_lst=[]#2
never_cnt_lst=[]#3
quite_cnt_lst=[]#4
but_cnt_lst=[]#5
very_cnt_lst=[]#6
more_cnt_lst=[]#7
br_cnt_lst=[]#8
else_cnt_lst=[]#9
must_cnt_lst=[]#10
many_cnt_lst=[]#11
up_cnt_lst=[]#12
so_cnt_lst=[]#13
all_cnt_lst=[]#14
dots_cnt_lst=[]#15
exclamation_mark_cnt_lst=[]#16
question_mark_cnt_lst=[]#17
not_cnt_lst=[]#18
er_cnt_lst=[]#19
est_cnt_lst=[]#20
ous_cnt_lst=[]#21
ly_cnt_lst=[]#22
dy_cnt_lst=[]#23
ful_cnt_lst=[]#24
ed_cnt_lst=[]#25
less_cnt_lst=[]#26
re_cnt_lst=[]#27
plus_cnt_lst=[]#28
minus_cnt_lst=[]#29
lol_cnt_lst=[]#30


CC_cnt_lst=[]#101
CD_cnt_lst=[]#102
DT_cnt_lst=[]#103
IN_cnt_lst=[]#104
JJ_cnt_lst=[]#105
JJR_cnt_lst=[]#106
JJS_cnt_lst=[]#107
MD_cnt_lst=[]#108
NN_cnt_lst=[]#109
NNS_cnt_lst=[]#110
NNP_cnt_lst=[]#111
NNPS_cnt_lst=[]#112
POS_cnt_lst=[]#113
PDT_cnt_lst=[]#114
PRP_cnt_lst=[]#115
PRP_dol_cnt_lst=[]#116
RB_cnt_lst=[]#117
RBR_cnt_lst=[]#118
RBS_cnt_lst=[]#119
TO_cnt_lst=[]#120
VB_cnt_lst=[]#121
VBD_cnt_lst=[]#122
VBG_cnt_lst=[]#123
VBN_cnt_lst=[]#124
VBP_cnt_lst=[]#125
VBZ_cnt_lst=[]#126
WDT_cnt_lst=[]#127
WP_cnt_lst=[]#128
WP_dol_cnt_lst=[]#129
WRB_cnt_lst=[]#130




pre_item=()
new_item=()

for tups in df_tup_lst:
#for tups in test_lst[0]:
    print(tups)
    
    #Counting Variables Initialization(Counting Specific Words in the segments) 
    repeat_word_cnt =0#1
    no_cnt =0#2
    never_cnt =0#3
    quite_cnt =0#4
    but_cnt =0#5
    very_cnt =0#6
    more_cnt =0#7
    br_cnt =0#8
    else_cnt =0#9
    must_cnt =0#10
    many_cnt =0#11
    up_cnt =0#12
    so_cnt =0#13
    all_cnt =0#14
    dots_cnt =0#15
    exclamation_mark_cnt =0#16
    question_mark_cnt =0#17
    not_cnt =0#18
    er_cnt =0#19
    est_cnt =0#20
    ous_cnt =0#21
    ly_cnt =0#22
    dy_cnt =0#23
    ful_cnt =0#24
    ed_cnt =0#25
    less_cnt =0#26
    re_cnt=0#27
    plus_cnt=0#28
    minus_cnt=0#29
    lol_cnt=0#30
    
    #Part-OF-Speech Tagging variable initialization
    CC_cnt =0#101
    CD_cnt =0#102
    DT_cnt =0#103
    IN_cnt =0#104
    JJ_cnt =0#105
    JJR_cnt =0#106
    JJS_cnt =0#107
    MD_cnt =0#108
    NN_cnt =0#109
    NNS_cnt =0#110
    NNP_cnt =0#111
    NNPS_cnt =0#112
    POS_cnt =0#113
    PDT_cnt =0#114
    PRP_cnt =0#115
    PRP_dol_cnt =0#116
    RB_cnt =0#117
    RBR_cnt =0#118
    RBS_cnt =0#119
    TO_cnt =0#120
    VB_cnt =0#121
    VBD_cnt =0#122
    VBG_cnt =0#123
    VBN_cnt =0#124
    VBP_cnt =0#125
    VBZ_cnt =0#126
    WDT_cnt =0#127
    WP_cnt =0#128
    WP_dol_cnt =0#129
    WRB_cnt =0#130
    
    
    for item in tups:
        pre_item=item
        
        #1
        #Check Consecutive repeated word 
        if new_item == pre_item:
            repeat_word_cnt+=1
        else:
            new_item = pre_item
        
        #Counting Specific Words in the segments
        
        #2
        if item[0].lower() == "no":
            print("no")
            no_cnt+=1
        #3    
        if item[0].lower() == "never":
            print("never")
            never_cnt+=1
        #4    
        if item[0].lower() == "quite":
            print("quite")
            quite_cnt+=1
        #5
        if item[0].lower() == "but":
            print("but")
            but_cnt+=1
    
        #6
        if item[0].lower() == "very":
            print("very")
            very_cnt+=1
        
        #7
        if item[0].lower() == "more":
            print("more")
            more_cnt+=1
                
        #8
        if item[0].lower() == "br":
            print("br")
            br_cnt+=1
        
        #9
        if item[0].lower() == "else":
            print("else")
            else_cnt+=1
        
        #10
        if item[0].lower() == "must":
            print("must")
            must_cnt+=1
        
        #11
        if item[0].lower() == "many":
            print("many")
            many_cnt+=1
                
        #12
        if item[0].lower() == "up":
            print("up")
            up_cnt+=1
        
        #13
        if item[0].lower() == "so":
            print("so")
            so_cnt+=1
        
        #14
        if item[0].lower() == "all":
            print("all")
            all_cnt+=1
        
        #15
        if item[0] == "...":
            print("...")
            dots_cnt+=1
        
        #16
        if item[0] == "!":
            print("!")
            exclamation_mark_cnt+=1
        
        #17
        if item[0] == "?":
            print("?")
            question_mark_cnt+=1
        
        #18
        if item[0] == "not":
            print("not")
            not_cnt+=1
            
            
        #Counting Specific pattern of last several characters from Adverbs and Adjuectives    
            
        #19
        if (item[0][-2:] == "er") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("er")
            er_cnt+=1
        #20     
        if (item[0][-3:] == "est") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("est")
            est_cnt+=1
        #21
        if (item[0][-4:] == "ous") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ous")
            ous_cnt+=1
        #22
        if (item[0][-2:] == "ly") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ly")
            ly_cnt+=1
        #23     
        if (item[0][-2:] == "dy") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("dy")
            dy_cnt+=1
        #24
        if (item[0][-2:] == "ful") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ful")
            ful_cnt+=1
        #25 
        if (item[0][-2:] == "ed") and ( item[1]=="VBD" or item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ed")
            ed_cnt+=1
        #26
        if (item[0][-4:] == "less") and ( item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("less")
            less_cnt+=1
        
        #27
        if (item[0][0:2] == "re"):
            print("re")
            re_cnt+=1
            
        #28
        for letter in item[0]:
            if letter is "+":
                plus_cnt+=1 
                
        #29
        for letter in item[0]:
            if letter is "-":
                minus_cnt+=1
        
        #30
        if item[0] is "lol":
            lol_cnt+=1#30
        
        print(item[1])
        #Counting  Part-Of-Speech
        #101
        if item[1] =="CC":
            print("CC")
            CC_cnt+=1
        #102
        if item[1] =="CD":
            print("CD")
            CD_cnt+=1
        #103
        if item[1] =="DT":
            print("DT")
            DT_cnt+=1
        #104
        if item[1] =="IN":
            print("IN")
            IN_cnt+=1
        #105
        if item[1] =="JJ":
            print("JJ")
            JJ_cnt+=1
        #106
        if item[1] =="JJR":
            print("JJR")
            JJR_cnt+=1
        #107
        if item[1] =="JJS":
            print("JJS")
            JJS_cnt+=1
        #108
        if item[1] =="MD":
            print("MD")
            MD_cnt+=1
        #109
        if item[1] =="NN":
            print("NN")
            NN_cnt+=1
        #110
        if item[1] =="NNS":
            print("NNS")
            NNS_cnt+=1
        #111
        if item[1] =="NNP":
            print("NNP")
            NNP_cnt+=1
        #112
        if item[1] =="NNPS":
            print("NNPS")
            NNPS_cnt+=1
        #113
        if item[1] =="POS":
            print("POS")
            POS_cnt+=1
        #114
        if item[1] =="PDT":
            print("PDT")
            PDT_cnt+=1
        #115
        if item[1] =="PRP":
            print("PRP")
            PRP_cnt+=1
        #116
        if item[1] =="PRP$":
            print("PRP$")
            PRP_dol_cnt+=1
        #117
        if item[1] =="RB":
            print("RB")
            RB_cnt+=1
        #118
        if item[1] =="RBR":
            print("RBR")
            RBR_cnt+=1
        #119
        if item[1] =="RBS":
            print("RBS")
            RBS_cnt+=1
        #120
        if item[1] =="TO":
            print("TO")
            TO_cnt+=1
        #121
        if item[1] =="VB":
            print("VB")
            VB_cnt+=1
        #122
        if item[1] =="VBD":
            print("VBD")
            VBD_cnt+=1
        #123
        if item[1] =="VBG":
            print("VBG")
            VBG_cnt+=1
        
        #124
        if item[1] =="VBN":
            print("VBN")
            VBN_cnt+=1
        #125
        if item[1] =="VBP":
            print("VBP")
            VBP_cnt+=1
        #126
        if item[1] =="VBZ":
            print("VBZ")
            VBZ_cnt+=1
        #127
        if item[1] =="WDT":
            print("WDT")
            WDT_cnt+=1
        #128
        if item[1] =="WP":
            print("WP")
            WP_cnt+=1
        #129
        if item[1] =="WP$":
            print("WP_dol")
            WP_dol_cnt+=1
        #130
        if item[1] =="WRB":
            print("WRB")
            WRB_cnt+=1
        
            
            
        

    repeat_word_cnt_lst.append(repeat_word_cnt)#1
    no_cnt_lst.append(no_cnt)#2
    never_cnt_lst.append(never_cnt)#3
    quite_cnt_lst.append(quite_cnt)#4
    but_cnt_lst.append(but_cnt)#5
    very_cnt_lst.append(very_cnt)#6
    more_cnt_lst.append(more_cnt)#7
    br_cnt_lst.append(br_cnt)#8
    else_cnt_lst.append(else_cnt)#9
    must_cnt_lst.append(must_cnt)#10
    many_cnt_lst.append(many_cnt)#11
    up_cnt_lst.append(up_cnt)#12
    so_cnt_lst.append(so_cnt)#13
    all_cnt_lst.append(all_cnt)#14
    dots_cnt_lst.append(dots_cnt)#15
    exclamation_mark_cnt_lst.append(exclamation_mark_cnt)#16
    question_mark_cnt_lst.append(question_mark_cnt)#17
    not_cnt_lst.append(not_cnt)#18
    er_cnt_lst.append(er_cnt)#19
    est_cnt_lst.append(est_cnt)#20
    ous_cnt_lst.append(ous_cnt)#21
    ly_cnt_lst.append(ly_cnt)#22
    dy_cnt_lst.append(dy_cnt)#23
    ful_cnt_lst.append(ful_cnt)#24
    ed_cnt_lst.append(ed_cnt)#25
    less_cnt_lst.append(less_cnt)#26
    re_cnt_lst.append(re_cnt) #27
    plus_cnt_lst.append(plus_cnt)#28
    minus_cnt_lst.append(minus_cnt)#29
    lol_cnt_lst.append(lol_cnt)#30


    CC_cnt_lst.append(CC_cnt)#101
    CD_cnt_lst.append(CD_cnt)#102
    DT_cnt_lst.append(DT_cnt)#103
    IN_cnt_lst.append(IN_cnt)#104
    JJ_cnt_lst.append(JJ_cnt)#105
    JJR_cnt_lst.append(JJR_cnt)#106
    JJS_cnt_lst.append(JJS_cnt)#107
    MD_cnt_lst.append(MD_cnt)#108
    NN_cnt_lst.append(NN_cnt)#109
    NNS_cnt_lst.append(NNS_cnt)#110
    NNP_cnt_lst.append(NNP_cnt)#111
    NNPS_cnt_lst.append(NNPS_cnt)#112
    POS_cnt_lst.append(POS_cnt)#113
    PDT_cnt_lst.append(PDT_cnt)#114
    PRP_cnt_lst.append(PRP_cnt)#115
    PRP_dol_cnt_lst.append(PRP_dol_cnt)#116
    RB_cnt_lst.append(RB_cnt)#117
    RBR_cnt_lst.append(RBR_cnt)#118
    RBS_cnt_lst.append(RBS_cnt)#119
    TO_cnt_lst.append(TO_cnt)#120
    VB_cnt_lst.append(VB_cnt)#121
    VBD_cnt_lst.append(VBD_cnt)#122
    VBG_cnt_lst.append(VBG_cnt)#123
    VBN_cnt_lst.append(NNPS_cnt)#124
    VBP_cnt_lst.append(POS_cnt)#125
    VBZ_cnt_lst.append(PDT_cnt)#126
    WDT_cnt_lst.append(PRP_cnt)#127
    WP_cnt_lst.append(PRP_dol_cnt)#128
    WP_dol_cnt_lst.append(RB_cnt)#129
    WRB_cnt_lst.append(RBR_cnt)#130

print(len(WRB_cnt_lst))

[('i', 'PRP'), ('found', 'VBD'), ('these', 'DT'), ('cookies', 'NNS'), ('really', 'RB'), ('moist', 'JJ'), ('and', 'CC'), ('yummy', 'NN'), ('.', '.')]
PRP
PRP
VBD
VBD
DT
DT
NNS
NNS
ly
re
RB
RB
JJ
JJ
CC
CC
NN
NN
.
[('i', 'PRP'), ('found', 'VBD'), ('these', 'DT'), ('cookies', 'NNS'), ('really', 'RB'), ('moist', 'JJ')]
PRP
PRP
VBD
VBD
DT
DT
NNS
NNS
ly
re
RB
RB
JJ
JJ
[('kids', 'NNS'), ('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
NNS
NNS
RB
RB
ed
VBD
VBD
DT
DT
NNS
NNS
[('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
RB
RB
ed
VBD
VBD
DT
DT
NNS
NNS
[('also', 'RB'), ('loved', 'VBD'), ('the', 'DT'), ('cookies', 'NNS')]
RB
RB
ed
VBD
VBD
DT
DT
NNS
NNS
[('these', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('absolutely', 'RB'), ('delicious', 'JJ'), (',', ',')]
DT
DT
NNS
NNS
VBP
VBP
ly
RB
RB
JJ
JJ
,
[('i', 'PRP'), ('received', 'VBP'), ('a', 'DT'), ('free', 'NNP'), ('cookie', 'NNP'), ('from', 'NNP'), ('influenster', 'NNP'), ('and', 'CC'), ('he', 'NNP'), ('to

.
[('they', 'PRP'), ('are', 'VBP'), ('very', 'RB'), ('soft', 'JJ')]
PRP
PRP
VBP
VBP
very
RB
RB
JJ
JJ
[('my', 'PRP$'), ('favorite', 'JJ'), ('grains', 'NNS')]
PRP$
PRP$
JJ
JJ
NNS
NNS
[('and', 'CC'), ('i', 'PRP'), ('was', 'VBD'), ('very', 'RB'), ('excited', 'JJ'), ('to', 'TO'), ('try', 'VB')]
CC
CC
PRP
PRP
VBD
VBD
very
RB
RB
ed
JJ
JJ
TO
TO
VB
VB
[('and', 'CC'), ('i', 'PRP'), ('was', 'VBD'), ('very', 'RB'), ('excited', 'JJ')]
CC
CC
PRP
PRP
VBD
VBD
very
RB
RB
ed
JJ
JJ
[('cookies', 'NNS'), (',', ','), ('especially', 'RB')]
NNS
NNS
,
ly
RB
RB
[('a', 'DT'), ('soft', 'JJ'), ('variety', 'NN'), ('.', '.')]
DT
DT
JJ
JJ
NN
NN
.
[('these', 'DT'), ('soft', 'JJ'), ('cookies', 'NNS'), ('are', 'VBP'), ('very', 'RB'), ('tasty', 'JJ')]
DT
DT
JJ
JJ
NNS
NNS
VBP
VBP
very
RB
RB
JJ
JJ
[('these', 'DT'), ('soft', 'JJ'), ('cookies', 'NNS')]
DT
DT
JJ
JJ
NNS
NNS
[('have', 'VBP'), ('a', 'DT'), ('great', 'JJ'), ('texture', 'NN'), ('.', '.')]
VBP
VBP
DT
DT
JJ
JJ
NN
NN
.
[('have', 'VBP'), ('a', 'DT'), ('great', 'JJ'), 

RB
RB
VB
VB
[('i', 'PRP'), ('gave', 'VBD'), ('it', 'PRP')]
PRP
PRP
VBD
VBD
PRP
PRP
[('i', 'PRP'), ('gave', 'VBD'), ('it', 'PRP')]
PRP
PRP
VBD
VBD
PRP
PRP
[('year', 'NN'), ('old', 'JJ'), ('son', 'NN'), (',', ',')]
NN
NN
JJ
JJ
NN
NN
,
[('old', 'JJ'), ('son', 'NN'), (',', ',')]
JJ
JJ
NN
NN
,
[('raisins', 'NNS'), ('or', 'CC'), ('anything', 'NN'), ('remotely', 'RB'), ('healthy', 'JJ'), (',', ','), ('he', 'PRP'), ('loved', 'VBD'), ('it', 'PRP'), ('.', '.')]
NNS
NNS
CC
CC
NN
NN
ly
re
RB
RB
JJ
JJ
,
PRP
PRP
ed
VBD
VBD
PRP
PRP
.
[('raisins', 'NNS'), ('or', 'CC'), ('anything', 'NN'), ('remotely', 'RB'), ('healthy', 'JJ'), (',', ',')]
NNS
NNS
CC
CC
NN
NN
ly
re
RB
RB
JJ
JJ
,
[('he', 'PRP'), ('loved', 'VBD'), ('it', 'PRP'), ('.', '.')]
PRP
PRP
ed
VBD
VBD
PRP
PRP
.
[('he', 'PRP'), ('loved', 'VBD'), ('it', 'PRP'), ('.', '.')]
PRP
PRP
ed
VBD
VBD
PRP
PRP
.
[('cookie', 'NNP'), ('is', 'VBZ'), ('great', 'JJ'), ('and', 'CC'), ('tasty', 'JJ'), ('.', '.')]
NNP
NNP
VBZ
VBZ
JJ
JJ
CC
CC
JJ
JJ
.
[('cookie', 'NNP'

NN
.
[('most', 'JJS'), ('cookies', 'NNS'), ('.', '.')]
JJS
JJS
NNS
NNS
.
[('these', 'DT'), ('are', 'VBP'), ('so', 'RB'), ('good', 'JJ'), ('and', 'CC')]
DT
DT
VBP
VBP
so
RB
RB
JJ
JJ
CC
CC
[('these', 'DT'), ('are', 'VBP'), ('so', 'RB'), ('good', 'JJ')]
DT
DT
VBP
VBP
so
RB
RB
JJ
JJ
[('combined', 'VBD'), ('was', 'VBD'), ('great', 'JJ'), ('!', '.'), ('!', '.')]
ed
VBD
VBD
VBD
VBD
JJ
JJ
!
.
!
.
[('all', 'DT'), ('look', 'VBP'), ('really', 'RB'), ('good', 'JJ'), ('.', '.')]
all
DT
DT
VBP
VBP
ly
re
RB
RB
JJ
JJ
.
[('these', 'DT'), ('were', 'VBD'), ('really', 'RB'), ('good', 'JJ'), ('especially', 'RB')]
DT
DT
VBD
VBD
ly
re
RB
RB
JJ
JJ
ly
RB
RB
[('like', 'VBP'), ('raisins', 'NNS'), ('.', '.')]
VBP
VBP
NNS
NNS
.
[('they', 'PRP'), ('taste', 'VBP'), ('very', 'RB'), ('fresh', 'JJ')]
PRP
PRP
VBP
VBP
very
RB
RB
JJ
JJ
[('everyone', 'NN'), ('loves', 'VBZ'), ('them', 'PRP'), ('!', '.')]
NN
NN
VBZ
VBZ
PRP
PRP
!
.
[('loves', 'VBZ'), ('them', 'PRP'), ('!', '.')]
VBZ
VBZ
PRP
PRP
!
.
[('loves', 'VBZ'), ('them',

NN
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
DT
DT
VBP
VBP
very
RB
RB
JJ
JJ
,
not
RB
RB
RB
RB
JJ
JJ
,
not
RB
RB
RB
RB
ly
RB
RB
,
ly
RB
RB
JJ
JJ
,
JJ
JJ
NNS
NNS
IN
IN
JJ
JJ
NN
NN
,
JJ
JJ
NN
NN
,
very
RB
RB
JJ
JJ
NN
NN
(
[('these', 'DT'), ('are', 'VBP'), ('very', 'RB'), ('good', 'JJ'), (',', ','), ('not', 'RB'), ('too', 'RB'), ('sweet', 'JJ'), (',', ','), ('not', 'RB'), ('too', 'RB'), ('crumbly', 'RB'), (',', ','), ('slightly', 'RB'), ('dry', 'JJ'), (',', ','), ('good', 'JJ'), ('raisins', 'NNS')]
DT
DT
VBP
VBP
very
RB
RB
JJ
JJ
,
not
RB
RB
RB
RB
JJ
JJ
,
not
RB
RB
RB
RB
ly
RB
RB
,
ly
RB
RB
JJ
JJ
,
JJ
JJ
NNS
NNS
[('low', 'JJ'), ('ratio', 'NN'), (',', ',')]
JJ
JJ
NN
NN
,
[('excellent', 'JJ'), ('texture', 'NN'), (',', ',')]
JJ
JJ
NN
NN
,
[('very', 'RB'), ('natural', 'JJ'), ('tasting', 'NN'), ('(', '(')]
very
RB
RB
JJ
JJ
NN
NN
(
[('im', 'NNP'), ('not', 'RB'), ('the', 'DT'), ('biggest', 'JJS'), ('fan', 'NN')]
NNP
NNP
not
RB
RB
DT
DT
est
JJS
JJS
NN
NN
[('the', 'DT'), ('biggest', 'JJS'), ('fan', 'N

...

RP
[('immediately', 'RB'), ('ate', 'VBD'), ('it', 'PRP'), ('up', 'RP')]
ly
RB
RB
VBD
VBD
PRP
PRP
up
RP
[('immediately', 'RB'), ('ate', 'VBD'), ('it', 'PRP')]
ly
RB
RB
VBD
VBD
PRP
PRP
[('he', 'PRP'), ('had', 'VBD'), ('ever', 'RB'), ('had', 'VBN')]
PRP
PRP
VBD
VBD
er
RB
RB
VBN
VBN
[('my', 'PRP$'), ('only', 'JJ'), ('criticism', 'NN')]
PRP$
PRP$
ly
JJ
JJ
NN
NN
[('is', 'VBZ'), ('that', 'IN'), ('even', 'RB')]
VBZ
VBZ
IN
IN
RB
RB
[('had', 'VBD'), ('raisins', 'NNS'), (',', ',')]
VBD
VBD
NNS
NNS
,
[('not', 'RB'), ('taste', 'VB'), ('them', 'PRP'), ('!', '.')]
not
RB
RB
VB
VB
PRP
PRP
!
.
[('not', 'RB'), ('taste', 'VB'), ('them', 'PRP'), ('!', '.')]
not
RB
RB
VB
VB
PRP
PRP
!
.
[('would', 'MD'), ('love', 'VB'), ('to', 'TO'), ('have', 'VB')]
MD
MD
VB
VB
TO
TO
VB
VB
[('they', 'PRP'), ('are', 'VBP'), ('always', 'RB'), ('soft', 'JJ')]
PRP
PRP
VBP
VBP
RB
RB
JJ
JJ
[('so', 'RB'), ('easy', 'JJ'), ('to', 'TO'), ('find', 'VB')]
so
RB
RB
JJ
JJ
TO
TO
VB
VB
[('and', 'CC'), ('it', 'PRP'), ('is', 'VBZ'), ('defint

VBD
so
RB
RB
JJ
JJ
CC
CC
NN
NN
.
[('these', 'DT'), ('cookies', 'NNS'), ('were', 'VBD'), ('so', 'RB'), ('soft', 'JJ')]
DT
DT
NNS
NNS
VBD
VBD
so
RB
RB
JJ
JJ
[('a', 'DT'), ('great', 'JJ'), ('snack', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('i', 'PRP'), ('received', 'VBD'), ('this', 'DT'), ('product', 'NN'), ('complimentary', 'NN')]
PRP
PRP
ed
re
VBD
VBD
DT
DT
NN
NN
NN
NN
[('i', 'PRP'), ("'m", 'VBP'), ('more', 'RBR'), ('apt', 'JJ')]
PRP
PRP
VBP
VBP
more
RBR
RBR
JJ
JJ
[('just', 'RB'), ('have', 'VB'), ('one', 'CD'), ('cookie', 'NN')]
RB
RB
VB
VB
CD
CD
NN
NN
[('way', 'NN'), ('rather', 'RB'), ('than', 'IN'), ('opening', 'VBG'), ('a', 'DT'), ('whole', 'JJ'), ('box', 'NN')]
NN
NN
er
RB
RB
IN
IN
VBG
VBG
DT
DT
JJ
JJ
NN
NN
[('way', 'NN'), ('rather', 'RB'), ('than', 'IN')]
NN
NN
er
RB
RB
IN
IN
[('opening', 'VBG'), ('a', 'DT'), ('whole', 'JJ'), ('box', 'NN')]
VBG
VBG
DT
DT
JJ
JJ
NN
NN
[('opening', 'VBG'), ('a', 'DT'), ('whole', 'JJ'), ('box', 'NN')]
VBG
VBG
DT
DT
JJ
JJ
NN
NN
[('a', 'DT'), ('whole', 'JJ'), ('box', 

NNP
NNP
NN
NN
NN
NN
[('my', 'PRP$'), ('voxbox', 'NNP'), ('and', 'CC'), ('was', 'VBD'), ('very', 'RB'), ('excited', 'JJ')]
PRP$
PRP$
NNP
NNP
CC
CC
VBD
VBD
very
RB
RB
ed
JJ
JJ
[('try', 'VB'), ('it', 'PRP'), ('.', '.')]
VB
VB
PRP
PRP
.
[('try', 'VB'), ('it', 'PRP'), ('.', '.')]
VB
VB
PRP
PRP
.
[('this', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('great', 'JJ')]
DT
DT
NNS
NNS
VBP
VBP
JJ
JJ
[('it', 'PRP'), ('in', 'IN'), ('ur', 'JJ'), ('bag', 'NN'), ('and', 'CC'), ('go', 'VB'), ('a++++++', 'NNP')]
PRP
PRP
IN
IN
JJ
JJ
NN
NN
CC
CC
VB
VB
NNP
NNP
[('it', 'PRP'), ('in', 'IN'), ('ur', 'JJ'), ('bag', 'NN')]
PRP
PRP
IN
IN
JJ
JJ
NN
NN
[('i', 'PRP'), ('do', 'VBP'), ('get', 'VB'), ('an', 'DT'), ('opportunity', 'NN')]
PRP
PRP
VBP
VBP
VB
VB
DT
DT
NN
NN
[('try', 'VB'), ('an', 'DT'), ('oatmeal', 'NN')]
VB
VB
DT
DT
NN
NN
[('try', 'VB'), ('an', 'DT'), ('oatmeal', 'NN')]
VB
VB
DT
DT
NN
NN
[('it', 'PRP'), ('is', 'VBZ'), ('usually', 'RB'), ('from', 'IN'), ('a', 'DT'), ('store-bought', 'JJ'), ('package', 'NN'),

VBP
JJ
JJ
[('i', 'PRP'), ('was', 'VBD'), ('given', 'VBN'), ('a', 'DT'), ('sample', 'NN')]
PRP
PRP
VBD
VBD
VBN
VBN
DT
DT
NN
NN
[('i', 'NN'), ('love', 'VBP'), ('oatmeal', 'NN'), (',', ',')]
NN
NN
VBP
VBP
NN
NN
,
[('love', 'VBP'), ('oatmeal', 'NN'), (',', ',')]
VBP
VBP
NN
NN
,
[('i', 'JJ'), ('love', 'VBP'), ('soft-baked', 'JJ'), ('cookies', 'NNS'), ('.', '.')]
JJ
JJ
VBP
VBP
ed
JJ
JJ
NNS
NNS
.
[('son', 'NN'), ('jordan', 'NNP'), ('is', 'VBZ'), ('autistic', 'JJ')]
NN
NN
NNP
NNP
VBZ
VBZ
JJ
JJ
[('he', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('very', 'RB'), ('picky', 'JJ'), ('eater', 'NN'), ('.', '.')]
PRP
PRP
VBZ
VBZ
DT
DT
very
RB
RB
JJ
JJ
NN
NN
.
[('a', 'DT'), ('very', 'RB'), ('picky', 'JJ'), ('eater', 'NN'), ('.', '.')]
DT
DT
very
RB
RB
JJ
JJ
NN
NN
.
[('these', 'DT'), ('are', 'VBP'), ('great', 'JJ'), ('and', 'CC')]
DT
DT
VBP
VBP
JJ
JJ
CC
CC
[('these', 'DT'), ('are', 'VBP'), ('great', 'JJ')]
DT
DT
VBP
VBP
JJ
JJ
[('something', 'NN'), ('else', 'RB'), ('less', 'RBR'), ('healthy', 'JJ'), ('.', '.')]


.
[('i', 'PRP'), ('will', 'MD'), ('definitely', 'RB'), ('be', 'VB')]
PRP
PRP
MD
MD
ly
RB
RB
VB
VB
[('buying', 'VBG'), ('this', 'DT'), ('again', 'RB'), ('!', '.')]
VBG
VBG
DT
DT
RB
RB
!
.
[('this', 'DT'), ('again', 'RB'), ('!', '.')]
DT
DT
RB
RB
!
.
[('i', 'PRP'), ('had', 'VBD'), ('a', 'DT'), ('problem', 'NN'), ('with', 'IN'), ('the', 'DT'), ('crumbly', 'NN'), ('texture', 'NN'), (',', ','), ('because', 'IN')]
PRP
PRP
VBD
VBD
DT
DT
NN
NN
IN
IN
DT
DT
NN
NN
NN
NN
,
IN
IN
[('i', 'PRP'), ('had', 'VBD'), ('a', 'DT'), ('problem', 'NN')]
PRP
PRP
VBD
VBD
DT
DT
NN
NN
[('i', 'PRP'), ('tried', 'VBD'), ('to', 'TO'), ('dip', 'VB')]
PRP
PRP
ed
VBD
VBD
TO
TO
VB
VB
[('the', 'DT'), ('cookie', 'NN'), ('into', 'IN'), ('a', 'DT'), ('glass', 'NN'), ('of', 'IN'), ('milk', 'NN'), (',', ','), ('a', 'DT'), ('big', 'JJ'), ('chunk', 'NN')]
DT
DT
NN
NN
IN
IN
DT
DT
NN
NN
IN
IN
NN
NN
,
DT
DT
JJ
JJ
NN
NN
[('a', 'DT'), ('big', 'JJ'), ('chunk', 'NN')]
DT
DT
JJ
JJ
NN
NN
[('it', 'PRP'), ('turned', 'VBD'), ('into', 'IN'), 

In [670]:
print(len(CC_cnt_lst))

6375


### Combining all the features into one Table and write to CSV

In [672]:
df_train = pd.DataFrame(list(zip(df_tup_lst,
                      df_syn0_lst,
                      df_syn_all_lst,
                     repeat_word_cnt_lst,
                        no_cnt_lst,
                        never_cnt_lst,
                        quite_cnt_lst,
                        but_cnt_lst,
                        very_cnt_lst,
                        more_cnt_lst,
                        br_cnt_lst,
                        else_cnt_lst,
                        must_cnt_lst,
                        many_cnt_lst,
                        up_cnt_lst,
                        so_cnt_lst,
                        all_cnt_lst,
                        dots_cnt_lst,
                        exclamation_mark_cnt_lst,
                        question_mark_cnt_lst,
                        not_cnt_lst,
                        er_cnt_lst,
                        est_cnt_lst,
                        ous_cnt_lst,
                        ly_cnt_lst,
                        dy_cnt_lst,
                        ful_cnt_lst,
                        ed_cnt_lst,
                        less_cnt_lst,
                        CC_cnt_lst,
                        CD_cnt_lst,
                        DT_cnt_lst,
                        IN_cnt_lst,
                        JJ_cnt_lst,
                        JJR_cnt_lst,
                        JJS_cnt_lst,
                        MD_cnt_lst,
                        NN_cnt_lst,
                        NNS_cnt_lst,
                        NNP_cnt_lst,
                        NNPS_cnt_lst,
                        POS_cnt_lst,
                        PDT_cnt_lst,
                        PRP_cnt_lst,
                        PRP_dol_cnt_lst,
                        RB_cnt_lst,
                        RBR_cnt_lst,
                        RBS_cnt_lst,
                        TO_cnt_lst,
                        VB_cnt_lst,
                        VBD_cnt_lst,
                        VBG_cnt_lst,
                        VBN_cnt_lst,
                        VBP_cnt_lst,
                        VBZ_cnt_lst,
                        WDT_cnt_lst,
                        WP_cnt_lst,
                        WP_dol_cnt_lst,
                        WRB_cnt_lst))
             , columns=["segments","syn0", "syn_all","repeat_word_cnt"
                                                            ,"no_cnt"
                                                            ,"never_cnt"
                                                            ,"quite_cnt"
                                                            ,"but_cnt"
                                                            ,"very_cnt"
                                                            ,"more_cnt"
                                                            ,"br_cnt"
                                                            ,"else_cnt"
                                                            ,"must_cnt"
                                                            ,"many_cnt"
                                                            ,"up_cnt"
                                                            ,"so_cnt"
                                                            ,"all_cnt"
                                                            ,"dots_cnt"
                                                            ,"exclamation_mark_cnt"
                                                            ,"question_mark_cnt"
                                                            ,"not_cnt"
                                                            ,"er_cnt"
                                                            ,"est_cnt"
                                                            ,"ous_cnt"
                                                            ,"ly_cnt"
                                                            ,"dy_cnt"
                                                            ,"ful_cnt"
                                                            ,"ed_cnt"
                                                            ,"less_cnt"


                                                            ,"CC_cnt"
                                                            ,"CD_cnt"
                                                            ,"DT_cnt"
                                                            ,"IN_cnt"
                                                            ,"JJ_cnt"
                                                            ,"JJR_cnt"
                                                            ,"JJS_cnt"
                                                            ,"MD_cnt"
                                                            ,"NN_cnt"
                                                            ,"NNS_cnt"
                                                            ,"NNP_cnt"
                                                            ,"NNPS_cnt"
                                                            ,"POS_cnt"
                                                            ,"PDT_cnt"
                                                            ,"PRP_cnt"
                                                            ,"PRP_dol_cnt"
                                                            ,"RB_cnt"
                                                            ,"RBR_cnt"
                                                            ,"RBS_cnt"
                                                            ,"TO_cnt"
                                                            ,"VB_cnt"
                                                            ,"VBD_cnt"
                                                            ,"VBG_cnt"
                                                            ,"VBN_cnt"
                                                            ,"VBP_cnt"
                                                            ,"VBZ_cnt"
                                                            ,"WDT_cnt"
                                                            ,"WP_cnt"
                                                            ,"WP_dol_cnt"
                                                            ,"WRB_cnt"])

#Create the Training Dataset
df_train.to_csv("final_train_table3.csv", index=False)

In [673]:
df_train.shape

(6375, 59)

## The following steps are run After manually labeling the class labels

## Train1 - 1065 rows

In [385]:
df_train= pd.read_csv("train1.csv", index_col=False)
print(df_train.shape)
df_train.head(3)

(1064, 60)


,class,segments,syn0,syn_all,repeat_word_cnt,no_cnt,never_cnt,quite_cnt,but_cnt,very_cnt,...,VB_cnt,VBD_cnt,VBG_cnt,VBN_cnt,VBP_cnt,VBZ_cnt,WDT_cnt,WP_cnt,WP_dol_cnt,WRB_cnt
0,1,"[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', ...",0.017857,0.045292,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,1,"[('these', 'DT'), ('soft', 'JJ'), ('oatmeal', ...",-0.093750,-0.105263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"[('i', 'PRP'), ('even', 'RB'), ('shared', 'VBD...",0.125000,0.012500,0,0,0,0,0,0,...,0,1,0,0,0,0,2,0,1,0


#### Convert the String Segment in training set back to tuples list

In [391]:

new_lst2=[]
for i in range(0,df_train.shape[0]):

    new_lst = []
    line = df_train.segments.iloc[i]
    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()
    
    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
    #print(new_lst)
    #print()
    new_lst2.append(new_lst)
len(new_lst2)

1064

### Classifier (Try 1)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [75]:
Y = np.array(df_train["class"])
X = np.array(df_train.iloc[:,2:])
print(X.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)

print(Counter(y_train))
print(Counter(y_test))


(1064, 58)
(1064,)
Counter({1: 549, 0: 163})
Counter({1: 253, 0: 99})


In [76]:
#Model
model = BernoulliNB()
model.fit(x_train,y_train)

#Prediction
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
pred_proba_test = model.predict_proba(x_test)

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

[[ 68  95]
 [ 45 504]]
Recall for Training Set:  0.9180327868852459
Precision for Training Set:  0.8414023372287145
Accuracy for Training Set:  0.8033707865168539

[[ 43  56]
 [ 23 230]]
Recall for Testing Set:  0.9090909090909091
Precision for Testing Set:  0.8041958041958042
Accuracy for Training Set:  0.7755681818181818


### Attempt to correct more "False Positive" classification to boost the Accuracy by Intensitify Polarity Score for Adverbs

#### Convert the String Segment in training set back to tuples list

In [332]:

new_lst2=[]
for i in range(0,df_train.shape[0]):

    new_lst = []
    line = df_train.segments.iloc[i]
    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()
    
    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
    #print(new_lst)
    #print()
    new_lst2.append(new_lst)

#### Calculate the Polarity Score with Adverb Booster

In [416]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
#df_lst = [0 for item in range(len(review_lst2))]
df_syn0_lst2 = []
df_syn_all_lst2 = []
df_tup_lst2 = []
ADV_booster=1.8 #Adv Booster Value
cnt=0



#for tups in review_lst[0:10]:
for tups in new_lst2:
    if len(tups)<3:
        continue
    #print(tups)
    lst=[]
    tot_pscore_syn0=0
    tot_pscore_syn_all = 0
    word_cnt=0
    
    for i in range(len(tups)):
        #print("tups[i]",tups[i])
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        #print("w",w)
        #print("tag",tag)
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print("1")
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                #print("2")
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                #print("3")
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            #print(synsets)
            #print(synset)
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            #print(swn_synset)
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            #print(pscore)
            
            #Adverb booster
            if wntag == wn.ADJ:
                pscore = pscore * ADV_booster
                #print(wntag)
            
            tot_pscore_syn0 += pscore
            
            tot_pscore_syn=0
            if len(synsets)>1:
                for synset in synsets:
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    #print(swn_synset)
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    #print("pscore",pscore)
                    
                    #Adverb booster
                    if wntag == wn.ADJ:
                        pscore = pscore * ADV_booster
                        #print(wntag)
                    
                    tot_pscore_syn += pscore
                tot_pscore_syn_all += (tot_pscore_syn/len(synsets))
                #print("tot_pscore_syn_all",tot_pscore_syn_all)
            else: tot_pscore_syn_all += tot_pscore_syn0
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
    
    if word_cnt == 0:
        df_syn0_lst2.append(0)
        df_syn_all_lst2.append(0)
        #print("Word count",word_cnt)
        df_tup_lst2.append(tups)
        cnt+=1
    else:
        df_syn0_lst2.append(tot_pscore_syn0/word_cnt)
        df_syn_all_lst2.append(tot_pscore_syn_all/word_cnt)
        #print("Word count",word_cnt)
        df_tup_lst2.append(tups)
        cnt+=1
        
print(len(df_syn0_lst2), len(df_syn_all_lst2), len(df_tup_lst2))
    



1064 1064 1064


### Classifier 2 (Adverb booster)

In [417]:
X2=X.copy()

#X2[:,2:].shape
lst1= [[item] for item in df_syn0_lst2]
lst2= [[item] for item in df_syn_all_lst2]

X2 = np.concatenate((X2[:,2:],lst1,lst2), axis=1)
print("Before edit Polarity by Booster",X.shape)
print("After edit Polarity by Booster",X2.shape)

Before edit Polarity by Booster (1064, 58)
After edit Polarity by Booster (1064, 58)


In [424]:
print(X2.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X2, Y, test_size=0.33, random_state=1)

print(Counter(y_train))
print(Counter(y_test))



#Model
model = BernoulliNB()
model.fit(x_train,y_train)

#Prediction
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
pred_proba_test = model.predict_proba(x_test)

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
prob_cutoff=0.4
pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
print(cm_test)

(1064, 58)
(1064,)
Counter({1: 549, 0: 163})
Counter({1: 253, 0: 99})
[[ 68  95]
 [ 45 504]]
Recall for Training Set:  0.9180327868852459
Precision for Training Set:  0.8414023372287145
Accuracy for Training Set:  0.8033707865168539

[[ 43  56]
 [ 23 230]]
Recall for Testing Set:  0.9090909090909091
Precision for Testing Set:  0.8041958041958042
Accuracy for Training Set:  0.7755681818181818
[[ 33  66]
 [ 15 238]]


## Train2 - 1515 rows

In [674]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec

In [477]:
df_train = pd.read_csv("train2.csv", index_col=False)
print(df_train.shape)
print(df_train.columns)
#df_train = df_train.loc[:,"syn0":'less_cnt']

(1514, 60)
Index(['class', 'segments', 'syn0', 'syn_all', 'repeat_word_cnt', 'no_cnt',
       'never_cnt', 'quite_cnt', 'but_cnt', 'very_cnt', 'more_cnt', 'br_cnt',
       'else_cnt', 'must_cnt', 'many_cnt', 'up_cnt', 'so_cnt', 'all_cnt',
       'dots_cnt', 'exclamation_mark_cnt', 'question_mark_cnt', 'not_cnt',
       'er_cnt', 'est_cnt', 'ous_cnt', 'ly_cnt', 'dy_cnt', 'ful_cnt', 'ed_cnt',
       'less_cnt', 'CC_cnt', 'CD_cnt', 'DT_cnt', 'IN_cnt', 'JJ_cnt', 'JJR_cnt',
       'JJS_cnt', 'MD_cnt', 'NN_cnt', 'NNS_cnt', 'NNP_cnt', 'NNPS_cnt',
       'POS_cnt', 'PDT_cnt', 'PRP_cnt', 'PRP_dol_cnt', 'RB_cnt', 'RBR_cnt',
       'RBS_cnt', 'TO_cnt', 'VB_cnt', 'VBD_cnt', 'VBG_cnt', 'VBN_cnt',
       'VBP_cnt', 'VBZ_cnt', 'WDT_cnt', 'WP_cnt', 'WP_dol_cnt', 'WRB_cnt'],
      dtype='object')


#### Convert the String Segment in training set back to tuples list

In [478]:

new_lst2=[]
for i in range(0,df_train.shape[0]):

    new_lst = []
    line = df_train.segments.iloc[i]
    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()
    
    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
    #print(new_lst)
    #print()
    new_lst2.append(new_lst)
len(new_lst2)

1514

### XXXXXXX Filtering out Segments not including Feature words

In [479]:
Y_true = []
X_true = []
for i in range(len(new_lst2)):
    if new_lst2[i][0][0] in feature_word_lst:
        Y_true.append(df_train["class"][i])
        X_true.append(df_train.loc[i,"segments":])
        
print(len(Y_true)        )
print(len(X_true)        )

1189
1189


### Before Adding POS

In [480]:
x_train[:,1:28]

array([[0.6130952381, 0, 0, ..., 0, 0, 0],
       [-0.1694444444, 0, 0, ..., 0, 0, 0],
       [-0.1015625, 0, 0, ..., 0, 0, 0],
       ...,
       [-0.2125, 0, 0, ..., 0, 0, 0],
       [-0.09375, 0, 0, ..., 0, 0, 0],
       [-0.11111111109999999, 0, 0, ..., 0, 0, 0]], dtype=object)

In [481]:
#Forming Class Labels and Other attributes
#Y = np.array(df_train["class"])
#X = np.array(df_train.loc[:,"syn0":'less_cnt'])
#X = np.array(df_train.loc[:,"syn0":])
Y=np.array(Y_true)
X=np.array(X_true)
print(X.shape)
print(Y.shape)

#Splitting Training and Testing Dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)



print(Counter(y_train))
print(Counter(y_test))
print()
#Model
#model = GradientBoostingClassifier( n_estimators = 100)
#model = BernoulliNB()
model = RandomForestClassifier(criterion = "entropy",   n_estimators = 500)
model.fit(x_train[:,1:28],y_train)

#Prediction
pred_train = model.predict(x_train[:,1:28])
pred_test = model.predict(x_test[:,1:28])
pred_proba_test = model.predict_proba(x_test[:,1:28])

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

(1189, 59)
(1189,)
Counter({1: 590, 0: 206})
Counter({1: 297, 0: 96})

[[171  35]
 [ 14 576]]
Recall for Training Set:  0.976271186440678
Precision for Training Set:  0.9427168576104746
Accuracy for Training Set:  0.9384422110552764

[[ 43  53]
 [ 35 262]]
Recall for Testing Set:  0.8821548821548821
Precision for Testing Set:  0.8317460317460318
Accuracy for Training Set:  0.7760814249363868


### After adding POS

In [622]:
#Forming Class Labels and Other attributes
#Y = np.array(df_train["class"])
#X = np.array(df_train.loc[:,"syn0":'less_cnt'])
#X = np.array(df_train.loc[:,"segments":])
Y=np.array(Y_true)
X=np.array(X_true)
print(X.shape)
print(Y.shape)
print(())

#Splitting Training and Testing Dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)

#x_train=x_train[:,1:]
#x_test=x_test[:,1:]


#Model
#model = GradientBoostingClassifier( n_estimators = 100)
#model = BernoulliNB()
model = RandomForestClassifier(criterion = "entropy",   n_estimators = 500)
model.fit(x_train[:,1:],y_train)

#Prediction
pred_train = model.predict(x_train[:,1:])
pred_test = model.predict(x_test[:,1:])
pred_proba_test = model.predict_proba(x_test[:,1:])

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

(1189, 59)
(1189,)
()
[[198   8]
 [  4 586]]
Recall for Training Set:  0.9932203389830508
Precision for Training Set:  0.9865319865319865
Accuracy for Training Set:  0.9849246231155779

[[ 47  49]
 [ 22 275]]
Recall for Testing Set:  0.9259259259259259
Precision for Testing Set:  0.8487654320987654
Accuracy for Training Set:  0.8193384223918575


## Filtering
### False Positive
### Relabel Class to 0 if the segments possess these positive words

In [623]:
for i in range(0, x_test.shape[0]):
    if x_test[i,2]<=-0.16 and pred_test[i]==1:
        pred_test[i]=0
        print(x_test[i,1])

cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_test)
print("Recall for testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for testing Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

-0.125
-0.1666666667
-0.375
-0.25
-0.25
-0.2
[[ 50  46]
 [ 25 272]]
Recall for testing Set:  0.9158249158249159
Precision for testing Set:  0.8553459119496856
Accuracy for testing Set:  0.8193384223918575


## Filtering
### Find similar Words in the segments related to Positive Sentiment Words (e.g. "like", "tasty")
### Relabel Class to 1 if the segments possess these positive words

In [534]:
from nltk.corpus import stopwords
stopword_lst = set(stopwords.words('english'))
#Punctuation list is implmented
punc_lst=[".",'?',"[","]",";","#","!","@",",","$",
          "%","^","&","*","(",")","_","=","-",
          "\"",":","~","'","+",">","<","/","|","{","}"]

#Word2Vec Model
w2v_model = Word2Vec(sent_word_lst)

word_lst=[]
feature_word_lst2=[]

#Find the feature words close to the following feature words
for word in ["great", "soft", "taste", "recommend", "good", "like","liked" ,"love", "loved",  "tasty"]:
#for word in ["tasty"]:
    word_lst += list(w2v_model.wv.most_similar(word, topn=60))

#Filtering Feature words
for tup in word_lst:
    if (tup[0] not in stopword_lst) and (tup[0] not in punc_lst) :
        feature_word_lst2.append(tup[0]) 


feature_word_lst2 = list(set(feature_word_lst2))
print("Number of Feature words",len(feature_word_lst2))
#feature_word_lst2

Number of Feature words 63


In [535]:
for i in range(0, x_test.shape[0]):
    for word in feature_word_lst2:
        #print(word)
        if (word in x_test[i][0]):
            #print("1",word)
            
            #print("2", pred_test[i])
            #print("3",x_test[i][0])
            pred_test[i]=1
            #break
        #else:
         #   print("3",x_test[i][0])
          #  print("4",pred_test[i])
           # pred_test[i]=0
            #break

cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_test)
print("Recall for testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for testing Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

[[ 25  71]
 [ 11 286]]
Recall for testing Set:  0.9629629629629629
Precision for testing Set:  0.8011204481792717
Accuracy for testing Set:  0.7913486005089059


## Train3 - 2000 rows

In [711]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec

In [680]:
df_train = pd.read_csv("train3.csv", index_col=False)
print(df_train.shape)
print(df_train.columns)
#df_train = df_train.loc[:,"syn0":'less_cnt']

(2011, 60)
Index(['class', 'segments', 'syn0', 'syn_all', 'repeat_word_cnt', 'no_cnt',
       'never_cnt', 'quite_cnt', 'but_cnt', 'very_cnt', 'more_cnt', 'br_cnt',
       'else_cnt', 'must_cnt', 'many_cnt', 'up_cnt', 'so_cnt', 'all_cnt',
       'dots_cnt', 'exclamation_mark_cnt', 'question_mark_cnt', 'not_cnt',
       'er_cnt', 'est_cnt', 'ous_cnt', 'ly_cnt', 'dy_cnt', 'ful_cnt', 'ed_cnt',
       'less_cnt', 'CC_cnt', 'CD_cnt', 'DT_cnt', 'IN_cnt', 'JJ_cnt', 'JJR_cnt',
       'JJS_cnt', 'MD_cnt', 'NN_cnt', 'NNS_cnt', 'NNP_cnt', 'NNPS_cnt',
       'POS_cnt', 'PDT_cnt', 'PRP_cnt', 'PRP_dol_cnt', 'RB_cnt', 'RBR_cnt',
       'RBS_cnt', 'TO_cnt', 'VB_cnt', 'VBD_cnt', 'VBG_cnt', 'VBN_cnt',
       'VBP_cnt', 'VBZ_cnt', 'WDT_cnt', 'WP_cnt', 'WP_dol_cnt', 'WRB_cnt'],
      dtype='object')


#### Convert the String Segment in training set back to tuples list

In [681]:

new_lst2=[]
for i in range(0,df_train.shape[0]):

    new_lst = []
    line = df_train.segments.iloc[i]
    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()
    
    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
    #print(new_lst)
    #print()
    new_lst2.append(new_lst)
len(new_lst2)

2011

### XXXXXXX Filtering out Segments not including Feature words

In [677]:
Y_true = []
X_true = []
for i in range(len(new_lst2)):
    if new_lst2[i][0][0] in feature_word_lst:
        Y_true.append(df_train["class"][i])
        X_true.append(df_train.loc[i,"segments":])
        
print(len(Y_true)        )
print(len(X_true)        )

1581
1581


### Before Adding POS

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [693]:
Y_true = []
X_true = []

X_true = df_train.loc[:,"segments":]
Y_true = df_train.loc[:,"class"]

print(len(X_true))
print(len(Y_true))
x_train[:,1:28]

2011
2011


array([[0.3125, 0.34375, 0, ..., 0, 0, 0],
       [0.0, 0.18333333329999998, 0, ..., 0, 0, 0],
       [-0.125, 0.1267857143, 0, ..., 0, 0, 0],
       ...,
       [0.0, 0.0, 0, ..., 0, 0, 0],
       [0.0, 0.1458333333, 0, ..., 0, 0, 0],
       [0.0, 0.0432692308, 0, ..., 0, 0, 1]], dtype=object)

In [751]:
#Forming Class Labels and Other attributes
#Y = np.array(df_train["class"])
#X = np.array(df_train.loc[:,"syn0":'less_cnt'])
#X = np.array(df_train.loc[:,"syn0":])
Y=np.array(Y_true)
X=np.array(X_true)
print(X.shape)
print(Y.shape)

#Splitting Training and Testing Dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)



print(Counter(y_train))
print(Counter(y_test))
print()
#Model
#model = KNeighborsClassifier(n_neighbors=3, metric="minkowski") 
#model = svm.SVC(kernel='linear', C=1000)
#model = GradientBoostingClassifier( n_estimators = 100)
#model = BernoulliNB()
#model = GaussianNB()
#model = RandomForestClassifier(criterion = "entropy",   n_estimators = 500)
model = RandomForestClassifier(criterion = "gini",
                             n_estimators = 100, 
                             max_features = 10, 
                             min_samples_split= 2,
                             min_samples_leaf=2)
model.fit(x_train[:,1:28],y_train)

#Prediction
pred_train = model.predict(x_train[:,1:28])
pred_test = model.predict(x_test[:,1:28])
#pred_proba_test = model.predict_proba(x_test[:,1:28])

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

(2011, 59)
(2011,)
Counter({1: 849, 0: 498})
Counter({1: 414, 0: 250})

[[411  87]
 [ 92 757]]
Recall for Training Set:  0.8916372202591284
Precision for Training Set:  0.8969194312796208
Accuracy for Training Set:  0.8671121009651076

[[164  86]
 [ 68 346]]
Recall for Testing Set:  0.8357487922705314
Precision for Testing Set:  0.8009259259259259
Accuracy for Training Set:  0.7680722891566265


### After adding POS

In [750]:
#Forming Class Labels and Other attributes
#Y = np.array(df_train["class"])
#X = np.array(df_train.loc[:,"syn0":'less_cnt'])
#X = np.array(df_train.loc[:,"segments":])
Y=np.array(Y_true)
X=np.array(X_true)
print(X.shape)
print(Y.shape)
print(())

#Splitting Training and Testing Dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, stratify=Y,random_state=1)

#x_train=x_train[:,1:]
#x_test=x_test[:,1:]


#Model
#model = KNeighborsClassifier(n_neighbors=3, metric="minkowski") 
#model = svm.SVC(kernel='linear', C=1000)
#model = GradientBoostingClassifier( n_estimators = 100)
#model = BernoulliNB()
#model = GaussianNB()
#model = RandomForestClassifier(criterion = "entropy",   n_estimators = 500)
model = RandomForestClassifier(criterion = "gini",
                             n_estimators = 100, 
                             max_features = 10, 
                             min_samples_split= 2,
                             min_samples_leaf=2)
model.fit(x_train[:,1:],y_train)

#Prediction
pred_train = model.predict(x_train[:,1:])
pred_test = model.predict(x_test[:,1:])
#pred_proba_test = model.predict_proba(x_test[:,1:])

#Confustion Matrix
cm_train = confusion_matrix(y_train, pred_train, labels=[0,1])
cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_train)
print("Recall for Training Set: ", cm_train[1,1]/sum(cm_train[1]))
print("Precision for Training Set: ", cm_train[1,1]/sum(cm_train[:,1]))
print("Accuracy for Training Set: ", (cm_train[1,1]+cm_train[0,0])/cm_train.sum())
print()
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

#print(pred_proba_test)
#prob_cutoff=0.5
#pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])
#cm_test = confusion_matrix(y_test, pred_test_by_prob, labels=[0,1])
#print(cm_test)

(2011, 59)
(2011,)
()
[[427  74]
 [ 27 819]]
Recall for Training Set:  0.9680851063829787
Precision for Training Set:  0.9171332586786114
Accuracy for Training Set:  0.9250185597624351

[[171  76]
 [ 48 369]]
Recall for Testing Set:  0.8848920863309353
Precision for Testing Set:  0.8292134831460675
Accuracy for Training Set:  0.8132530120481928


### Tuning Classifier parameter

In [731]:
from sklearn.metrics import accuracy_score

#Function run the classifier
def calc_params(x_train,x_test,y_train, clf):
    model=clf
    model = model.fit(x_train,y_train)
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    pred_proba_test = model.predict_proba(x_test)
    return pred_train, pred_test, pred_proba_test


#Function to calculate the accuracy of classifier
def calc_accuracy(pred_train, pred_test, y_train, y_test):
    train_score = accuracy_score(y_train, pred_train)
    test_score = accuracy_score(y_test, pred_test)
    return train_score, test_score

In [746]:
Y=np.array(Y_true)
X=np.array(X_true.iloc[:,1:])
#############################
n_est_lst=[]
cv_mean_train_score_lst = []
cv_mean_test_score_lst = []
cv_mean_specificity_lst = []
cv_mean_sensitivity_lst = []
cv_mean_precision_lst = []
time_lst=[]
n_est_lst2=[]
cutoff_lst2=[]
max_feature_lst2=[]
cv_mean_NO_of_pred_1_lst=[]
NO_of_pred_1_lst=[]
criterion_lst2=[]
min_sample_split_lst2=[]
warm_start_lst2=[]
min_sample_leaf_lst2=[]
##############################
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import neighbors, tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import BaggingClassifier
from sklearn.cross_validation import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
##################################
K=3 #Set the number of Fold Cross Validataion
#cv = KFold(len(X),K,shuffle=True)

skf = StratifiedKFold(n_splits=K, shuffle=True)
#########################################
#alpha_lst = [0.2,0.4,0.6,0.8,1.0]
#warm_start_lst=[True, False]
n_est_lst= [100,200,500]
#cutoff_lst= list(np.linspace(0.75,0.9,8))
max_feature_lst = [4,6,8,10]
min_sample_split_lst = [2,3,4]
criterion_lst=["gini", "entropy"]
min_sample_leaf_lst = [2,3]

########################################
for n_est in n_est_lst:
    start=time.time()
    for max_feature in max_feature_lst:
        #for cutoff in cutoff_lst:
        for min_sample_split in min_sample_split_lst:
            for criterion in criterion_lst:
                for min_sample_leaf in min_sample_leaf_lst:
                    print("min_sample is ",min_sample_leaf)
                    #start=time.time()
                    n_est_lst2.append(n_est)
                    #print(n_est_lst2)
                    #cutoff_lst2.append(cutoff)
                    #print(cutoff_lst2)
                    max_feature_lst2.append(max_feature)
                    #print(max_feature_lst2)
                    min_sample_split_lst2.append(min_sample_split)
                    criterion_lst2.append(criterion)
                    #warm_start_lst2.append(warm_start)
                    min_sample_leaf_lst2.append(min_sample_leaf)
        ################################################### Classifier            
                    #clf = BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(criterion='entropy'),n_estimators=n_est)
                    #clf = BaggingClassifier(base_estimator=BernoulliNB(),n_estimators=n_est)
                    #clf = tree.DecisionTreeClassifier(criterion="entropy")
                    #clf = BernoulliNB()
                    #clf = GaussianNB()
                    #n_neighbors=3
                    #clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance',metric='minkowski')
                    #clf = SVC(C=0.001, kernel ="linear" )#"poly",degree=3)
                    #clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=10,criterion = criterion, max_features = max_feature, min_samples_split= min_sample_split,min_samples_leaf=min_sample_leaf),n_estimators=n_est, learning_rate=0.5)
                    #clf = AdaBoostClassifier(GaussianNB(),n_estimators=n_est)
                    clf = RandomForestClassifier(criterion = criterion, 
                                                 n_estimators = n_est, 
                                                 max_features = max_feature, 
                                                 min_samples_split= min_sample_split,
                                                 min_samples_leaf=min_sample_leaf )
                    #clf = GradientBoostingClassifier(criterion = criterion, n_estimators = n_est, max_features = max_feature, min_samples_split= min_sample_split,min_samples_leaf=min_sample_leaf )
        ################################################## Reset list value
                    train_score_lst = []
                    test_score_lst = []
                    specificity_lst = []
                    sensitivity_lst = []
                    precision_lst= []
                    NO_of_pred_1_lst=[]
        ################################################### Cross-Validation
                    #for train_index, test_index in skf.split(X, Y):
                    for train, test in skf.split(X, Y):
                    #for i , (train,test) in enumerate(cv):
                        #Plug in the "calc_params" function
                        pred_train, pred_test, pred_proba_test = calc_params([X[k] for k in train],
                                                            [X[k] for k in test],
                                                            Y[train], 
                                                            clf)
                        #prob_cutoff=cutoff
                        #pred_test_by_prob = np.array([1 if prob>=prob_cutoff else 0 for prob in pred_proba_test[:,1] ])

                        #plug in the "calc_accuracy" function
                        #train_score, test_score = calc_accuracy(pred_train, pred_test, y_train, y_test)
                        train_score, test_score = calc_accuracy(pred_train,  pred_test, Y[train], Y[test])

                        #plug in the "calc_accuracy" function
                        #train_score, test_score = calc_accuracy(pred_train, pred_test, Y[train], Y[test])

                        #Store Accuracy of Training and Testing into lists
                        train_score_lst.append(train_score)
                        test_score_lst.append(test_score)


                        from sklearn.metrics import confusion_matrix
                        #tree_cm = confusion_matrix(Y[test], pred_test, labels=[0,1])
                        tree_cm = confusion_matrix(Y[test], pred_test, labels=[0,1])                
                        #print(tree_cm)
                        specificity_lst.append(tree_cm[0,0] / (tree_cm[0,0]+tree_cm[0,1]))
                        sensitivity_lst.append(tree_cm[1,1] / (tree_cm[1,0]+tree_cm[1,1]))
                        precision_lst.append(tree_cm[1,1] / (tree_cm[0,1]+tree_cm[1,1]))
                        NO_of_pred_1_lst.append(tree_cm[1,1])

                    cv_mean_NO_of_pred_1_lst.append(round(np.mean(NO_of_pred_1_lst)))
                    cv_mean_train_score_lst.append(np.mean(train_score_lst))
                    cv_mean_test_score_lst.append(np.mean(test_score_lst))
                    cv_mean_specificity_lst.append(np.mean(specificity_lst))
                    cv_mean_sensitivity_lst.append(np.mean(sensitivity_lst))
                    cv_mean_precision_lst.append(np.mean(precision_lst))        
            ################################################### End Cross-Validation
    end=time.time()
    #time_lst.append(start-end)
    print(end-start)
    print()

acc_table= pd.DataFrame(list(zip(n_est_lst2,
                                 #cutoff_lst2,
                                 max_feature_lst2,
                                 min_sample_split_lst2,
                                 criterion_lst2,
                                 #warm_start_lst2,
                                 min_sample_leaf_lst2,
                                 
                                 cv_mean_NO_of_pred_1_lst,
                                 cv_mean_precision_lst,
                                 cv_mean_train_score_lst,
                                 cv_mean_test_score_lst ,
                                 cv_mean_specificity_lst,
                                 cv_mean_sensitivity_lst,)),
                        columns=("n_estimators",
                                 #"cutoff",
                                 "max_feature",
                                 "min_sample_split",
                                 "criterion",
                                 #"warm_start",
                                 "min_sample_leaf",
                                 
                                 "NO_inst_Pred_1",
                                 "cv_mean_precision_lst",
                                 "cv_mean_train_score" ,
                                 "cv_mean_test_score" ,
                                 "cv_mean_specificity",
                                 "cv_mean_sensitivity", ))
print(acc_table)
print("0",len(n_est_lst2))
print("1",len( max_feature_lst2))
print("2",len( min_sample_split_lst2))
print( "3",len(criterion_lst2))
print( "4",len(min_sample_leaf_lst2))
print( "5",len(cv_mean_NO_of_pred_1_lst))
print( "6",len(cv_mean_precision_lst))
print( "7",len(cv_mean_train_score_lst))
print( "8",len(cv_mean_test_score_lst ))
print( "9",len(cv_mean_specificity_lst))
print("10",len( cv_mean_sensitivity_lst))

##Write the result the above test result to csv file
acc_table.to_csv("RF_result1.csv", index=False)
print("Done!!!")

min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
36.30323505401611

min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample is  3
min_sample is  2
min_sample i

In [748]:
result_df = pd.read_csv("RF_result1.csv", index_col=False)
result_df=result_df.dropna(axis=0, how="any")
#result_df=result_df[result_df["NO_inst_Pred_1"]>10]
result_df=result_df[result_df["cv_mean_precision_lst"]>0.80]
result_df.sort_values(by=["cv_mean_test_score", "cv_mean_sensitivity","cv_mean_precision_lst"], ascending=[False,False,False]).head(5)

,n_estimators,max_feature,min_sample_split,criterion,min_sample_leaf,NO_inst_Pred_1,cv_mean_precision_lst,cv_mean_train_score,cv_mean_test_score,cv_mean_specificity,cv_mean_sensitivity
36,100,10,2,gini,2,364.0,0.827999,0.917952,0.802085,0.696535,0.864608
134,500,10,2,entropy,2,359.0,0.832552,0.923670,0.799609,0.709885,0.852732
128,500,8,4,gini,2,362.0,0.827476,0.906767,0.798606,0.696514,0.859066
76,200,8,3,gini,2,361.0,0.826880,0.908752,0.798117,0.696552,0.858274
136,500,10,3,gini,2,361.0,0.825698,0.918448,0.797114,0.693863,0.858274


## Filtering
### False Positive
### Relabel Class to 0 if the segments possess these positive words

In [690]:
for i in range(0, x_test.shape[0]):
    if x_test[i,2]<=-0.16 and pred_test[i]==1:
        pred_test[i]=0
        print(x_test[i,1])

cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_test)
print("Recall for testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for testing Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

-0.1875
-0.1666666667
-0.1666666667
-0.3333333333
[[164  86]
 [ 54 360]]
Recall for testing Set:  0.8695652173913043
Precision for testing Set:  0.8071748878923767
Accuracy for testing Set:  0.7891566265060241


## Filtering
### Find similar Words in the segments related to Positive Sentiment Words (e.g. "like", "tasty")
### Relabel Class to 1 if the segments possess these positive words

In [691]:
from nltk.corpus import stopwords
stopword_lst = set(stopwords.words('english'))
#Punctuation list is implmented
punc_lst=[".",'?',"[","]",";","#","!","@",",","$",
          "%","^","&","*","(",")","_","=","-",
          "\"",":","~","'","+",">","<","/","|","{","}"]

#Word2Vec Model
w2v_model = Word2Vec(sent_word_lst)

word_lst=[]
feature_word_lst2=[]

#Find the feature words close to the following feature words
for word in ["great", "soft", "taste", "recommend", "good", "like","liked" ,"love", "loved",  "tasty"]:
#for word in ["tasty"]:
    word_lst += list(w2v_model.wv.most_similar(word, topn=60))

#Filtering Feature words
for tup in word_lst:
    if (tup[0] not in stopword_lst) and (tup[0] not in punc_lst) :
        feature_word_lst2.append(tup[0]) 


feature_word_lst2 = list(set(feature_word_lst2))
print("Number of Feature words",len(feature_word_lst2))
#feature_word_lst2

Number of Feature words 63


In [692]:
for i in range(0, x_test.shape[0]):
    for word in feature_word_lst2:
        #print(word)
        if (word in x_test[i][0]):
            #print("1",word)
            
            #print("2", pred_test[i])
            #print("3",x_test[i][0])
            pred_test[i]=1
            #break
        #else:
         #   print("3",x_test[i][0])
          #  print("4",pred_test[i])
           # pred_test[i]=0
            #break

cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_test)
print("Recall for testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for testing Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

[[ 95 155]
 [ 13 401]]
Recall for testing Set:  0.9685990338164251
Precision for testing Set:  0.7212230215827338
Accuracy for testing Set:  0.7469879518072289


### Confidence Interval for the best Model

In [726]:
from sklearn.metrics import accuracy_score

n_times=100 #Preset 100 times trial 
train_score_lst=[] #Preset the Train score list
test_score_lst=[] #Preset the Test score list

Y=np.array(Y_true)
X=np.array(X_true)

for i in range(1,n_times+1):

    #Split the train set, test set, train class, test class
    from sklearn.cross_validation import train_test_split
    x_train,x_test,y_train,y_test = train_test_split(X, #Predictor Variables
                                                     Y, #Class labels
                                                     stratify=Y, #data is split in a stratified fashion, using this as the class labels
                                                     test_size=0.34)#Test size is set as 34% of entire set
                                                     #random_state=i) # Set the randon state for spliting data  in same order
    #Random Forest Classifier
    model = RandomForestClassifier(criterion = "entropy",   n_estimators = 500)
    

    #Plug in the train dataset with target class
    model = model.fit(x_train[:,1:], y_train)
    #Predict y value on testing data
    y_pre = model.predict(x_test[:,1:])

    #Calculate the Accuracy Score of the Training and Testing Set
    train_score_lst.append(model.score(x_train[:,1:], y_train))
    test_score_lst.append(accuracy_score(y_test, y_pre))
    



In [727]:
train_score_lst_DT = train_score_lst
test_score_lst_DT = test_score_lst

train_mean=np.mean(np.array(train_score_lst))
test_mean=np.mean(np.array(test_score_lst))

print("Decision Tree Result:")
print("Mean of Accuracy of Training Data is ",round(train_mean*100,2),'%')
print("Mean of Accuracy of Testing Data is ",round(test_mean*100,2),'%')

Decision Tree Result:
Mean of Accuracy of Training Data is  98.97 %
Mean of Accuracy of Testing Data is  79.75 %


In [728]:
train_var = np.var(np.array(train_score_lst))
test_var = np.var(np.array(test_score_lst))

print("Decision Tree Result:")
print("Variance of Accuracy of Training Data is ",train_var)
print("Variance of Accuracy of Testing Data is ",test_var)

Decision Tree Result:
Variance of Accuracy of Training Data is  3.70349968681302e-06
Variance of Accuracy of Testing Data is  0.0002080332409972297


In [729]:
#p,q, n values setting
p=train_mean
q=1-p
n=len(x_train)

print("Decision Tree Result:")

#95% C.I.
z=1.96
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("95 Percent Confidence Interval of Training Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

#90% C.I.
z=1.65
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("90 Percent Confidence Interval of Training Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

#80%C.I.
z=1.28
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("80 Percent Confidence Interval of Training Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

Decision Tree Result:
95 Percent Confidence Interval of Training Data is (0.984,0.995)
90 Percent Confidence Interval of Training Data is (0.985,0.994)
80 Percent Confidence Interval of Training Data is (0.986,0.993)


In [730]:
#p,q, n values setting
p=test_mean
q=1-p
n=len(x_test)

print("Naive Bayes Result:")

#95% C.I.
z=1.96
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("95 Percent Confidence Interval of Testing Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

#90% C.I.
z=1.65
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("90 Percent Confidence Interval of Testing Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

#80%C.I.
z=1.28
ci_left, ci_right = (p)-(z*np.sqrt(p*q/n)), (p)+(z*np.sqrt(p*q/n))
print("80 Percent Confidence Interval of Testing Data is (%0.3f,%0.3f)"%(ci_left,ci_right))

Naive Bayes Result:
95 Percent Confidence Interval of Testing Data is (0.767,0.828)
90 Percent Confidence Interval of Testing Data is (0.772,0.823)
80 Percent Confidence Interval of Testing Data is (0.778,0.817)


# End Classifier

In [536]:
for i in range(0, pred_test.shape[0]):
    if y_test[i] == 0 and pred_test[i]==1:
        print(i ,x_test[i][0])

1 [('just', 'RB'), ('baked', 'VBD'), ('it', 'PRP'), ('.', '.')]
2 [('a', 'DT'), ('big', 'JJ'), ('glass', 'NN')]
6 [('the', 'DT'), ('ingredients', 'NNS'), ('are', 'VBP'), ('in', 'IN'), ('very', 'RB'), ('tiny', 'JJ'), ('print', 'NN'), (',', ',')]
12 [('oatmeal', 'JJ'), ('raisin', 'NN'), ('cookies', 'NNS'), ('.', '.')]
16 [('i', 'PRP'), ('am', 'VBP'), ('very', 'RB'), ('picky', 'JJ'), ('about', 'IN'), ('my', 'PRP$'), ('oatmeal', 'JJ'), ('raisin', 'NN')]
23 [('say', 'VBP'), ('that', 'IN'), ('it', 'PRP')]
27 [('not', 'RB'), ('forget', 'VB'), ('the', 'DT'), ('wholesome', 'NN'), ('ingredients', 'NNS'), ('it', 'PRP')]
31 [('not', 'RB'), ('just', 'RB'), ('texture', 'NN'), ('.', '.')]
33 [('not', 'RB'), ('tell', 'VB'), ('your', 'PRP$'), ('kids', 'NNS'), ('they', 'PRP')]
41 [('these', 'DT'), ('too', 'RB'), ('and', 'CC')]
42 [('i', 'PRP'), ('decided', 'VBD'), ('to', 'TO'), ('split', 'VB')]
45 [('cookie', 'NN'), ('just', 'RB'), ('because', 'IN')]
53 [('the', 'DT'), ('taste', 'NN'), ('is', 'VBZ'), ('

In [12]:
for i in range(0, pred_test.shape[0]):
    if pred_test[i]==1:
        if "not" in x_test[i][0] or "too" in x_test[i][0] or "n\'t" in x_test[i][0]: #or "also" in x_test[i][0]:
            pred_test[i] = 0

        


cm_test = confusion_matrix(y_test, pred_test, labels=[0,1])
print(cm_test)
print("Recall for Testing Set: ", cm_test[1,1]/sum(cm_test[1]))
print("Precision for Testing Set: ", cm_test[1,1]/sum(cm_test[:,1]))
print("Accuracy for Training Set: ", (cm_test[1,1]+cm_test[0,0])/cm_test.sum())

[[ 86  54]
 [ 39 321]]
Recall for Testing Set:  0.8916666666666667
Precision for Testing Set:  0.856
Accuracy for Training Set:  0.814


## Aspect-based opinion summary on Positive Reviews

#### Convert the String Segment in training set back to tuples list

In [65]:
#Pick out all the segments with Class 1 (Positive Reviews)
segment_lst = df_train.segments[df_train["class"] == 1]

new_lst3 = []

for line in segment_lst:
    new_lst=[]

    #print(line)
    #print()
    line = line[3:-3]
    line = line.replace('"',"'")
    #print(line)
    line = line.replace("'), ('","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.replace("', '","ashgh582E@#$$")
    #print(line)
    #print()
    line = line.split("ashgh582E@#$$")
    #print(line)
    #print()

    j=0
    while j < len(line):
        new_lst.append(tuple((line[j],line[j+1])))
        j+=2
        
    new_lst3.append(new_lst)
    i+=1
print(len(new_lst3))    

1101


### Word_Tokenize from the POS Tuples

In [424]:
sent_word_lst=[]
for line in new_lst3:
    new_lst=[]
    for item in line:
        #print(item[0])
        new_lst.append(item[0])
    sent_word_lst += [new_lst]

print("Number of Positive Reviews: ",len(new_lst4))
sent_word_lst[0:5]

Number of Positive Reviews:  1101


[['these',
  'soft',
  'oatmeal',
  'raisin',
  'cookies',
  'are',
  'actually',
  'quite',
  'good',
  '.'],
 ['these', 'soft', 'oatmeal', 'raisin', 'cookies'],
 ['i', 'even', 'shared', 'it'],
 ['i', 'even', 'shared', 'it'],
 ['i', "'m", 'a', 'pretty', 'big', 'health', 'nut']]

In [131]:
sent_pos_lst=[]
for line in new_lst3:
    new_lst=[]
    for item in line:
        #print(item[0])
        new_lst.append(item[1])
    sent_pos_lst += new_lst

print("Number of Positive Reviews: ",len(new_lst4))
sent_pos_lst[0:20]

Number of Positive Reviews:  1101


['DT',
 'JJ',
 'NN',
 'NN',
 'NNS',
 'VBP',
 'RB',
 'RB',
 'JJ',
 '.',
 'DT',
 'JJ',
 'NN',
 'NN',
 'NNS',
 'PRP',
 'RB',
 'VBD',
 'PRP',
 'PRP']

### Unigrams in Positive Reviews

In [132]:
unigrams=[]
for line in sent_word_lst:
    unigrams += line
    
fduni = nltk.FreqDist(unigrams)

#Sort out the top 200 unigrams from reviews
count=0
top_uni_lst=[]
for k in sorted(fduni, key=fduni.get, reverse=True):
    top_uni_lst.append(k)
    count+=1
    if count>100:
        break
top_uni_lst

['.',
 'a',
 'i',
 'the',
 'it',
 ',',
 'cookies',
 '!',
 'cookie',
 'soft',
 'are',
 'was',
 'they',
 'and',
 'oatmeal',
 'this',
 'these',
 'great',
 'my',
 'is',
 'good',
 'loved',
 'snack',
 'love',
 'to',
 'them',
 'so',
 'quaker',
 'taste',
 'delicious',
 'not',
 '/',
 '<',
 'br',
 '>',
 'baked',
 'very',
 'were',
 'really',
 'that',
 'you',
 'for',
 'try',
 'more',
 'chewy',
 'raisin',
 'too',
 'also',
 'buy',
 'we',
 'perfect',
 'in',
 'will',
 'sample',
 'quick',
 'received',
 'yummy',
 'sweet',
 'like',
 'of',
 'with',
 'kids',
 'had',
 'got',
 'treat',
 'flavor',
 'homemade',
 'have',
 'definitely',
 'healthy',
 'free',
 'even',
 'go',
 'just',
 'best',
 'healthier',
 'be',
 'individually',
 'make',
 'raisins',
 'enjoyed',
 'an',
 "'s",
 'tasty',
 'product',
 'would',
 'daughter',
 'nice',
 'she',
 'he',
 'but',
 'texture',
 'much',
 'eating',
 'as',
 "'re",
 'whole',
 '...',
 'bad',
 'on',
 'am']

### Bigrams in Postiive Reviews

In [133]:
for line in sent_word_lst:
    bigrams += list(nltk.bigrams(line))

fdbi = nltk.FreqDist(bigrams)

#Sort out the top 200 unigrams from reviews
count=0
top_bi_lst=[]
for k in sorted(fdbi, key=fdbi.get, reverse=True):
    top_bi_lst.append(k)
    count+=1
    if count>100:
        break
top_bi_lst

[('this', 'cookie'),
 ('<', 'br'),
 ('br', '/'),
 ('/', '>'),
 ('it', 'was'),
 ('they', 'are'),
 ('cookies', 'are'),
 ('these', 'cookies'),
 ('a', 'great'),
 ('oatmeal', 'cookies'),
 ('soft', 'baked'),
 ('baked', 'oatmeal'),
 ('cookies', '.'),
 ('soft', ','),
 ('i', 'love'),
 ('oatmeal', 'raisin'),
 ('the', 'cookie'),
 ('a', 'good'),
 ('it', '.'),
 ('quaker', 'soft'),
 ('the', 'cookies'),
 ('it', 'is'),
 ('>', '<'),
 ('loved', 'it'),
 ('cookie', 'was'),
 ('i', 'received'),
 ('cookie', '.'),
 ('!', '!'),
 ('is', 'a'),
 ('loved', 'them'),
 ('i', 'was'),
 ('a', 'quick'),
 ('soft', 'and'),
 ('raisin', 'cookies'),
 ('it', "'s"),
 ('good', '.'),
 ('oatmeal', 'cookie'),
 ('i', 'got'),
 ('the', 'best'),
 ('was', 'soft'),
 ('loved', 'the'),
 ('was', 'so'),
 ('was', 'a'),
 ('it', '!'),
 ('are', 'so'),
 ('chewy', ','),
 ('love', 'them'),
 ('cookies', '!'),
 ('so', 'good'),
 ('cookies', ','),
 ('and', 'it'),
 ('they', 'taste'),
 ('these', 'are'),
 ('very', 'soft'),
 ('this', 'product'),
 ('them', 

array([["[('i', 'PRP'), ('would', 'MD'), ('definitely', 'RB'), ('recommend', 'VB')]",
        0.0625, 0.25, ..., 0, 1, 0],
       ["[('amazon', 'NNP'), ('has', 'VBZ'), ('it', 'PRP')]", 0.125,
        0.03125, ..., 0, 0, 0],
       ["[('not', 'RB'), ('entirely', 'RB'), ('dismiss', 'VB'), ('them', 'PRP'), ('either', 'DT'), ('.', '.')]",
        -0.0416666667, -0.125, ..., 0, 2, 0],
       ...,
       ["[('tastes', 'VBZ'), ('homemade', 'NN'), ('!', '.')]", -0.0625,
        0.03125, ..., 0, 0, 0],
       ["[('<', 'JJ'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP')]",
        0.0, 0.0, ..., 0, 0, 0],
       ["[('toddler', 'NN'), ('and', 'CC'), ('she', 'PRP'), ('also', 'RB'), ('loved', 'VBD'), ('it', 'PRP'), (',', ',')]",
        0.1666666667, 0.19791666670000002, ..., 0, 1, 0]], dtype=object)

### Features Distribution (Adverbs / Adjective ending letters) 

In [92]:
dots_cnt=0
er_cnt=0
est_cnt=0
ous_cnt=0
ly_cnt=0
dy_cnt=0
ful_cnt=0
ed_cnt=0
less_cnt=0


for item in unigrams:
    if "..." in item:
        dots_cnt += 1
    if "er" in item:
        er_cnt += 1
    if "est" in item:
        est_cnt +=1
    if "ous" in item:
        ous_cnt +=1
    if "ly" in item:
        ly_cnt +=1
    if "dy" in item:
        dy_cnt+=1
    if "ful" in item:
        ful_cnt +=1
    if "ed" in item:
        ed_cnt +=1
    if "less" in item:
        less_cnt +=1
        
print(dots_cnt,
er_cnt,
est_cnt,
ous_cnt,
ly_cnt,
dy_cnt,
ful_cnt,
ed_cnt,
less_cnt)

10 295 28 48 134 5 8 248 4


In [101]:
CC_cnt =0#101
CD_cnt =0#102
DT_cnt =0#103
IN_cnt =0#104
JJ_cnt =0#105
JJR_cnt =0#106
JJS_cnt =0#107
MD_cnt =0#108
NN_cnt =0#109
NNS_cnt =0#110
NNP_cnt =0#111
NNPS_cnt =0#112
POS_cnt =0#113
PDT_cnt =0#114
PRP_cnt =0#115
PRP_dol_cnt =0#116
RB_cnt =0#117
RBR_cnt =0#118
RBS_cnt =0#119
TO_cnt =0#120
VB_cnt =0#121
VBD_cnt =0#122
VBG_cnt =0#123
VBN_cnt =0#124
VBP_cnt =0#125
VBZ_cnt =0#126
WDT_cnt =0#127
WP_cnt =0#128
WP_dol_cnt =0#129
WRB_cnt =0#130

for item in pos_new_lst:
    if "CC" in item:
        CC_cnt +=1#101
    if "CD" in item:
        CD_cnt +=1#102
    if "DT" in item:
        DT_cnt +=1#103
    if "IN" in item:
        IN_cnt +=1#104
    if "JJ" in item:
        JJ_cnt =0#105
    if "JJR" in item:
        JJR_cnt +=1#106
    if "JJS" in item:    
        JJS_cnt +=1#107
    if "MD" in item:
        MD_cnt +=1#108
    if "NN" in item:
        NN_cnt +=1#109
    if "NNS" in item:
        NNS_cnt +=1#110
    if "NNP" in item:
        NNP_cnt +=1#111
    if "NNPS" in item:
        NNPS_cnt +=0#112
    if "POS" in item:
        POS_cnt +=1#113
    if "PDT" in item:
        PDT_cnt +=1#114
    if "PRP" in item:
        PRP_cnt +=1#115
    if "PRP$" in item:
        PRP_dol_cnt +=1#116
    if "RB" in item:
        RB_cnt +=1#117
    if "RBR" in item:
        RBR_cnt +=1#118
    if "RBS" in item:
        RBS_cnt +=1#119
    if "TO" in item:
        TO_cnt +=1#120
    if "VB" in item:
        VB_cnt +=1#121
    if "VBD" in item:
        VBD_cnt +=1#122
    if "VBG" in item:
        VBG_cnt +=1#123
    if "VBN" in item:
        VBN_cnt +=1#124
    if "VBP" in item:
        VBP_cnt +=1#125
    if "VBZ" in item:
        VBZ_cnt +=1#126
    if "WDT" in item:
        WDT_cnt +=1#127
    if "WP" in item:
        WP_cnt +=1#128
    if "WP$" in item:
        WP_dol_cnt +=1#129
    if "WRB" in item:
        WRB_cnt +=1#130
print(RB_cnt, VB_cnt)

416 1033


# END !!!!

In [91]:

dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
dog.path_similarity(cat)

0.2

In [161]:
syn_good = wn.synsets("good", pos=wntag)[0]
syn_declicious = wn.synsets("delicious")[0]
syn_excellent = wn.synsets("excellent")[0]
syn_great = wn.synsets("great", pos=wntag)[0]

syn_w = wn.synsets("bad", pos=wntag)[0]
#syn_w = wn.synsets("delicious", pos=wntag)[0]

syn_good.path_similarity(syn_great)
#syn_excellent
#syn_great
#syn_good

syn_good_n = wn.synsets("good", pos=wntag)[0]
syn_good_a = wn.synsets("good", pos=wntag)[4]
syn_good_s = wn.synsets("good", pos=wntag)[10]

wn.synsets("delicious")

#wn.synsets("excellent", pos=wntag)
wn.synsets("excellent")

[Synset('excellent.s.01')]

In [63]:

import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
label_lst = []
avg_label_score_lst = []


cp = nltk.RegexpParser(grammar)


start=time.time()
token_line_lst = []
cnt=0

sents=["I love this food. I really like it"]

for line in sents:
    token_line_lst=[]
    print(line)
    for token_line in sent_tokenize(line):
        print(token_line)
        token_line_lst.append(token_line)
    
    sentlist = [word_tokenize(line) for line in token_line_lst]
    print(sentlist)
    
    review_lst=[]    
    for sent in sentlist: #Iterate the Review sentlist 
        tree = cp.parse(nltk.pos_tag(sent)) #pos_tag the sentence and parse the sentence
        #print(tree)
        for subtree in tree.subtrees():
            if subtree.label() =="NP":
                #print("NP")
                #print(subtree.leaves())    
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1a":
                #print("clause1a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1b":
                #print("clause1b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2a":
                #print("clause2a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2b":
                #print("clause2b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause3":
                #print("clause3")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause4":
                #print("clause4")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause5":
                #print("clause5")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
        
    #Filtering out all the segments which do not have feature words              
    review_lst2=[] #initialize a new list
    for line in review_lst: #iterating the list with extracted segments
        for item in line:
            if item[0] in feature_lst: #Feature list is obtained in Step 3
                review_lst2.append(line) #Append all the segments have feature words in the new list
                continue
        
    

##################
    #print(len(review_lst))
    #print(review_lst)
    #print("# of segments",len(review_lst2))
    #print(review_lst2)
##################
    df_lst1=[]
    df_lst2=[]

    for tups in review_lst2:
        lst=[]
        tot_pscore=0
        word_cnt=0
        
        for i in range(len(tups)):
            tups[i]=list(tups[i]) #Convert the list back to tuples
            w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
            tag=tups[i][1] #Assign the tag from the 2nd element of tuples

            #If the word is a negative contraction suffix ('negcontractions'), 
            #change it to the word: "not"
            if w in negcontractions:
                w="not"
                
            # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
            wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
            if wntag is not None and (w not in stopwords or w in negwords): 
                #print(w)
                tups[i][0] = w 
                word_cnt+=1
                lemma = wn_lemmatize(w, wntag) #lemmatize token

                if lemma is None:
                    tups[i]=tuple(tups[i])
                    continue

                synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

                if not synsets:
                    tups[i]=tuple(tups[i])
                    continue

                # take the first sense
                synset = synsets[0]
                swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                tot_pscore+=pscore

                #Convert the list back to tuples
                tups[i]=tuple(tups[i])

            else:
                tups[i][0] = w 
                tups[i]=tuple(tups[i])

        if word_cnt == 0: df_lst1.append(0)
        else: df_lst1.append(tot_pscore/word_cnt)
        #print("df_lst1:",df_lst1)


        df_lst2.append(tups)

    
    if np.mean(df_lst1) > 0 :
        label=1
    else:
        label=0
    #print(df_lst2)
    avg_label_score_lst.append(np.mean(df_lst1))
    label_lst.append(label)
   
    print(cnt)
    cnt+=1
end=time.time()
print("%d, Time taken %f min"%(cnt+1, ((end-start)/ 60)))
#print("avg",avg_label_score_lst)    
#print("label",label_lst)
print(len(label_lst))

I love this food. I really like it
I love this food.
I really like it
[['I', 'love', 'this', 'food', '.'], ['I', 'really', 'like', 'it']]
0
2, Time taken 0.000164 min
1


/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [119]:
avg_label_score_lst

NameError: name 'avg_label_score_lst' is not defined

In [131]:
y__true = df.Score.copy()
#y__pred = label_lst
y__true[y__true < 3] =0
y__true[y__true > 3] =1


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y__true,y__pred,labels=[0,1,3])
cm
#(cm[i,i] / sum(cm[i]))

array([[ 52429,  29608,      0],
       [223949, 219828,      0],
       [ 22287,  20353,      0]])

In [130]:
Counter(y__true)




Counter({0: 82037, 1: 443777, 3: 42640})

In [116]:
len(label_lst)
df.Score.head(5)

0    5
1    1
2    4
3    2
4    5
Name: Score, dtype: int64

In [105]:
df_pred = df.copy()
df_pred.shape

(568454, 10)

In [112]:
pred_col = pd.DataFrame(label_lst, columns=["pred"])
#pd.concat((df_pred, pred_col))
pred_col

,pred
0,1
1,0
2,1
3,0
4,0
5,1
6,0
7,1
8,0
9,1


### Step-6: Polarity Score

##### Given Functions

In [ ]:
w="comfortable"
tag="JJ"
wntag = to_wntag(tag)
lemma = wn_lemmatize(w, wntag)
synsets = wn.synsets(lemma, pos=wntag)
print(synsets)
synset = synsets[2]


swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
print(swn_synset)
pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
pscore

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
df_lst = [0 for item in range(len(review_lst2))]
df_lst2 = []
df_lst1 = []

cnt=0
for tups in review_lst2:
    lst=[]
    tot_pscore=0
    word_cnt=0
    for i in range(len(tups)):
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print(w)
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            tot_pscore+=pscore
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
            
    df_lst1.append(tot_pscore/word_cnt)
    df_lst2.append(tups)
    cnt+=1
    
    
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
df2= pd.DataFrame(df_lst2)
df = pd.concat((df1,df2), axis=1)
df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 
                    ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

In [ ]:
len(review_lst2)

In [ ]:
pre_lst=[]
new_lst=[]
cnt=0
with open("XXXX-rooms.txt", "w") as f:
    for row in range(0,df.shape[0]):
        new_lst = list(df.iloc[row])
        new_lst = [item  for item in new_lst if item is not None]
        #print(new_lst)

        if pre_lst == new_lst:
            continue
            
        for item in df.iloc[row]:
            if item is not None:
                try:
                    if type(float(item)) == float:
                        #print(item, end=", [")
                        f.write("{0}, [".format(item))
                        continue
                except:
                    pass
                #print(item, end=",")
                f.write("{0}, ".format(item))

            else:
                #print("]",end="\n")
                f.write("]\n")
                cnt+=1
                break
        pre_lst = new_lst
f.close()
print(cnt)    

########################END

In [ ]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
sent = nltk.pos_tag(nltk.word_tokenize("quick"))
print(sent)

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<JJ>*<NN>} 
FOO:{<NP><VB.><JJ>}"""
#"NP:{DT|PRP\$>?<JJ>*<NN>}" #, FOO: {<NP><VB.><JJ>}"

#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)



In [ ]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)


In [ ]:
for subtree in tree.subtrees():
    if subtree.label() =="NP":
        print(subtree.leaves())

In [ ]:
for sent in sentlist:
    cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print(subtree.leaves())